# Strongest Paper Experiments for ISMIR 2026

This notebook contains all experiments needed to make the strongest possible paper for ISMIR 2026 submission.

## Experiment Structure

### PHASE A: Configuration Optimization (RUN FIRST)
Determine the optimal training configuration. All subsequent experiments use the winning combination.

- **A1: Fold Method Comparison** - piece-based vs performer-based vs stratified
  - Piece-based: Same performer can appear in train/test (current approach)
  - Performer-based: Each performer in one fold only (tests generalization to unseen performers)
  - Stratified: Balanced by composer/difficulty (reduces variance)
  
- **A2: Audio Source Comparison** - Salamander vs Pianoteq ensemble
  - Salamander: 1202 samples, single timbre
  - Pianoteq ensemble: 7212 samples (6 soundfonts), varied timbres

### PHASE B: Core Validation
Using optimal config from Phase A:

- **B1: Multi-Seed Stability** - 5 seeds x 4 folds, report mean +/- std
- **B2: Cross-Soundfont Generalization** - Train on 5 soundfonts, test on held-out 6th
- **B3: Statistical Rigor** - Bootstrap CIs, significance tests vs baselines

### PHASE C: Cross-Dataset Transfer
Test generalization to external datasets:

- **C1: PSyllabus Difficulty** - Correlation with external piano difficulty ratings
- **C2: ASAP Multi-Performer** - Same pieces played by different performers
- **C3: MAESTRO Zero-Shot** - Transfer to MAESTRO piano dataset

### PHASE D: Advanced Analysis
- **D1: Per-Dimension Breakdown** - R2 for each of 19 PercePiano dimensions
- **D2: Error Analysis** - By composer, difficulty level, segment position

## Available Audio Sources
- **Salamander Grand Piano**: 1202 samples (original renders)
- **Pianoteq Renders** (6 soundfonts x 1202 = 7212 samples):
  - `HB_Steinway_Model_D`, `YC5_Vintage`, `K2_Basic`
  - `NY_Steinway_D_Honky_Tonk`, `NY_Steinway_D_Worn_Out`, `U4_Small`

## Requirements
- **Compute**: Thunder Compute A100 (80GB VRAM)
- **Storage**: rclone configured with `gdrive:` remote

In [ ]:
# Cell 1: CUDA setup (must be before any CUDA operations)
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:   
    raise RuntimeError("GPU required for training")

In [ ]:
# Cell 2: Install rclone
!curl -fsSL https://rclone.org/install.sh | sudo bash 2>&1 | grep -E "(successfully|already)" || echo "rclone installed"

In [ ]:
# Cell 3: Install dependencies and clone repo
!pip install transformers librosa soundfile pytorch_lightning nnAudio scipy scikit-learn muq requests tqdm --quiet

import os
REPO_DIR = '/tmp/crescendai'
if os.path.exists(REPO_DIR):
    !cd {REPO_DIR} && git pull origin main
else:
    !git clone https://github.com/jai-dhiman/crescendai.git {REPO_DIR}

print(f"Repo: {REPO_DIR}")

In [ ]:
# Cell 4: Imports
import sys
sys.path.insert(0, f'{REPO_DIR}/model/src')

import json
import subprocess
import warnings
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Tuple, Any, Optional

import numpy as np
import torch
import pytorch_lightning as pl
from scipy import stats
from sklearn.metrics import r2_score, mean_absolute_error
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from tqdm.auto import tqdm

from audio_experiments import PERCEPIANO_DIMENSIONS, DIMENSION_CATEGORIES, BASE_CONFIG, SEED
from audio_experiments.extractors import extract_muq_embeddings, MuQExtractor
from audio_experiments.models import MuQStatsModel, StatsPoolingModel
from audio_experiments.data import MERTDataset, mert_collate_fn
from audio_experiments.training import (
    run_4fold_mert_experiment,
    should_run_experiment,
    sync_experiment_to_gdrive,
    get_completed_experiments,
    print_experiment_status,
    bootstrap_r2_extended,
    compute_comprehensive_metrics,
)
from audio_experiments.training.sync import numpy_serializer

warnings.filterwarnings('ignore')
torch.set_float32_matmul_precision('medium')

print(f"PyTorch: {torch.__version__}")
print(f"Imports: OK")

In [ ]:
# Cell 5: Path configuration
DATA_ROOT = Path('/tmp/strongest_paper_experiments')
AUDIO_DIR = DATA_ROOT / 'audio'
LABEL_DIR = DATA_ROOT / 'labels'
MUQ_CACHE_ROOT = DATA_ROOT / 'muq_cache'
CHECKPOINT_ROOT = DATA_ROOT / 'checkpoints'
RESULTS_DIR = DATA_ROOT / 'results'
LOG_DIR = DATA_ROOT / 'logs'
FIGURES_DIR = RESULTS_DIR / 'figures'

# Cross-dataset directories
MAESTRO_DIR = DATA_ROOT / 'maestro'

# Pianoteq soundfont directories
PIANOTEQ_ROOT = DATA_ROOT / 'pianoteq'
PIANOTEQ_SOUNDFONTS = [
    'HB_Steinway_Model_D',
    'YC5_Vintage',
    'K2_Basic',
    'NY_Steinway_D_Honky_Tonk',
    'NY_Steinway_D_Worn_Out',
    'U4_Small',
]

# GDrive paths
GDRIVE_AUDIO = 'gdrive:crescendai_data/audio_baseline/percepiano_rendered'
GDRIVE_PIANOTEQ = 'gdrive:crescendai_data/audio_baseline/pianoteq_rendered'
GDRIVE_LABELS = 'gdrive:crescendai_data/percepiano_labels'
GDRIVE_FOLDS = 'gdrive:crescendai_data/percepiano_fold_assignments.json'
GDRIVE_MUQ_CACHE = 'gdrive:crescendai_data/audio_baseline/muq_embeddings'
GDRIVE_PHASE2_RESULTS = 'gdrive:crescendai_data/checkpoints/audio_phase2'
GDRIVE_RESULTS = 'gdrive:crescendai_data/checkpoints/strongest_paper'

for d in [AUDIO_DIR, LABEL_DIR, MUQ_CACHE_ROOT, CHECKPOINT_ROOT,
          RESULTS_DIR, LOG_DIR, FIGURES_DIR, MAESTRO_DIR, PIANOTEQ_ROOT]:
    d.mkdir(parents=True, exist_ok=True)

def run_rclone(cmd, desc=""):
    """Run rclone command with error handling."""
    if desc:
        print(f"{desc}...")
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        raise RuntimeError(f"rclone failed: {desc}\nCommand: {' '.join(cmd)}\nStderr: {result.stderr}")
    return result

# Check rclone
result = subprocess.run(['rclone', 'listremotes'], capture_output=True, text=True)
if 'gdrive:' not in result.stdout:
    raise RuntimeError("rclone 'gdrive' not configured")

print(f"Data root: {DATA_ROOT}")
print(f"GDrive results: {GDRIVE_RESULTS}")
print(f"Pianoteq soundfonts: {PIANOTEQ_SOUNDFONTS}")

In [ ]:
# Cell 6: Download data
run_rclone(['rclone', 'copy', GDRIVE_AUDIO, str(AUDIO_DIR), '--progress'], "Downloading audio")
run_rclone(['rclone', 'copy', GDRIVE_LABELS, str(LABEL_DIR)], "Downloading labels")

FOLD_FILE = DATA_ROOT / 'folds.json'
run_rclone(['rclone', 'copyto', GDRIVE_FOLDS, str(FOLD_FILE)], "Downloading folds")

# Load labels and folds
LABEL_FILE = LABEL_DIR / 'label_2round_mean_reg_19_with0_rm_highstd0.json'
with open(LABEL_FILE) as f:
    LABELS = json.load(f)
with open(FOLD_FILE) as f:
    FOLD_ASSIGNMENTS = json.load(f)

# Create key->fold_id mapping
FOLD_BY_KEY = {}
for fold_id in range(4):
    for key in FOLD_ASSIGNMENTS.get(f"fold_{fold_id}", []):
        FOLD_BY_KEY[key] = fold_id

ALL_KEYS = sorted(FOLD_BY_KEY.keys())
print(f"Samples per fold: {[len(FOLD_ASSIGNMENTS.get(f'fold_{i}', [])) for i in range(4)]}")
print(f"Total samples: {len(ALL_KEYS)}")
print(f"Audio files: {len(list(AUDIO_DIR.glob('*.wav')))}")

# Initialize results tracking
ALL_RESULTS = {}

In [ ]:
# Cell 7: Initialize results tracking and check completed experiments
ALL_RESULTS = {}

# Get completed experiments from GDrive to enable resume/skip
print("Checking GDrive for completed experiments...")
COMPLETED_CACHE = get_completed_experiments(GDRIVE_RESULTS)
print(f"Found {len(COMPLETED_CACHE)} completed experiments on GDrive")

if COMPLETED_CACHE:
    print("\nCompleted experiments:")
    for exp_id, r2 in sorted(COMPLETED_CACHE.items()):
        print(f"  {exp_id}: R2={r2:.4f}")

# Define all experiment IDs for this notebook (in execution order)
EXPERIMENT_IDS = [
    # PHASE A: Configuration Optimization
    'A1a_piece_fold',           # Piece-based folds (current)
    'A1b_performer_fold',       # Performer-based folds
    'A1c_stratified_fold',      # Stratified by composer/difficulty
    'A2_pianoteq_ensemble',     # All 6 Pianoteq soundfonts
    
    # PHASE B: Core Validation
    'B1_multi_seed_stability',  # 5 seeds x 4 folds
    'B2_cross_soundfont',       # Leave-one-out soundfont
    'B3_bootstrap_significance',# Bootstrap CIs + significance tests
    
    # PHASE C: Cross-Dataset Transfer
    'C1_psyllabus_difficulty',  # PSyllabus difficulty correlation
    'C2_asap_multiperformer',   # ASAP multi-performer analysis
    'C3_maestro_transfer',      # MAESTRO zero-shot
    
    # PHASE D: Advanced Analysis
    'D1_per_dimension',         # Per-dimension R2 breakdown
    'D2_error_analysis',        # By composer, difficulty, position
]

print_experiment_status(EXPERIMENT_IDS, COMPLETED_CACHE)

---
## PHASE A: Configuration Optimization

**Run these experiments FIRST.** Results determine the optimal training configuration for all subsequent experiments.

### A1: Fold Method Comparison

Compare three fold assignment strategies on Salamander audio:

| Method | Description | What it tests |
|--------|-------------|---------------|
| **Piece-based** | Random split by piece (current) | Standard CV |
| **Performer-based** | Each performer in one fold only | Generalization to unseen performers |
| **Stratified** | Balanced by composer + difficulty | Reduced variance |

### A2: Audio Source Comparison

Using the best fold method from A1, compare training data:

| Source | Samples | Description |
|--------|---------|-------------|
| **Salamander** | 1,202 | Single soundfont baseline |
| **Pianoteq ensemble** | 7,212 | All 6 soundfonts (implicit augmentation) |

In [ ]:
# Cell 8: Download MuQ L9-12 embeddings
MUQ_L9_12_DIR = MUQ_CACHE_ROOT / 'L9-12'
MUQ_L9_12_DIR.mkdir(parents=True, exist_ok=True)

GDRIVE_MUQ_L9_12 = 'gdrive:crescendai_data/audio_baseline/muq_embeddings/L9-12'

# Try to download cached embeddings
result = subprocess.run(['rclone', 'lsf', GDRIVE_MUQ_L9_12], capture_output=True, text=True)
if result.returncode == 0 and result.stdout.strip():
    print("Downloading cached MuQ L9-12 embeddings...")
    run_rclone(['rclone', 'copy', GDRIVE_MUQ_L9_12, str(MUQ_L9_12_DIR), '--progress'],
               "Downloading MuQ L9-12 embeddings")
else:
    print("No cached MuQ L9-12 embeddings found. Will extract from audio.")

# Check what we have
cached_keys = {p.stem for p in MUQ_L9_12_DIR.glob('*.pt')}
missing_keys = [k for k in ALL_KEYS if k not in cached_keys]
print(f"MuQ L9-12 Cached: {len(cached_keys)}, Missing: {len(missing_keys)}")

# Extract missing embeddings
if missing_keys:
    print(f"\nExtracting {len(missing_keys)} MuQ L9-12 embeddings...")
    extract_muq_embeddings(AUDIO_DIR, MUQ_L9_12_DIR, missing_keys, layer_start=9, layer_end=13)
    
    # Upload newly extracted embeddings
    print("\nUploading MuQ L9-12 embeddings to GDrive...")
    run_rclone(['rclone', 'copy', str(MUQ_L9_12_DIR), GDRIVE_MUQ_L9_12],
               "Uploading MuQ L9-12 embeddings")

In [ ]:
# Cell 9: Define seeds and model configuration for multi-seed stability
STABILITY_SEEDS = [42, 123, 456, 789, 1337]

# M1c_muq_L9-12 configuration (best performing model)
M1C_CONFIG = {
    **BASE_CONFIG,
    'input_dim': 1024,
    'hidden_dim': 512,
    'dropout': 0.2,
    'learning_rate': 1e-4,
    'weight_decay': 1e-5,
    'pooling_stats': 'mean_std',
}

def make_muq_stats_model(cfg):
    """Factory function for MuQ stats model."""
    return MuQStatsModel(
        input_dim=cfg['input_dim'],
        hidden_dim=cfg['hidden_dim'],
        dropout=cfg['dropout'],
        learning_rate=cfg['learning_rate'],
        weight_decay=cfg['weight_decay'],
        pooling_stats=cfg['pooling_stats'],
        max_epochs=cfg['max_epochs'],
    )

print(f"Seeds to test: {STABILITY_SEEDS}")
print(f"Configuration: hidden_dim={M1C_CONFIG['hidden_dim']}, lr={M1C_CONFIG['learning_rate']}")

In [ ]:
# A1a: Piece-based fold baseline (current approach)
exp_id = 'A1a_piece_fold'

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    exp_checkpoint_dir = CHECKPOINT_ROOT / exp_id
    exp_checkpoint_dir.mkdir(parents=True, exist_ok=True)
    
    pl.seed_everything(42, workers=True)
    
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: {exp_id}")
    print(f"Description: MuQ L9-12 with piece-based folds (current methodology)")
    print(f"Audio: Salamander | Folds: Piece-based | Samples: {len(ALL_KEYS)}")
    print(f"{'='*70}")
    
    existing_folds = [f for f in range(4) if (exp_checkpoint_dir / f"fold{f}_best.ckpt").exists()]
    if existing_folds:
        print(f"Found existing checkpoints for folds: {existing_folds}")
    
    existing_results_file = RESULTS_DIR / f'{exp_id}.json'
    fold_results = {}
    if existing_results_file.exists():
        with open(existing_results_file) as f:
            existing_data = json.load(f)
            if 'fold_results' in existing_data:
                fold_results = {int(k): v for k, v in existing_data['fold_results'].items()}
    
    all_preds, all_labels = [], []
    
    for fold in range(4):
        ckpt_path = exp_checkpoint_dir / f"fold{fold}_best.ckpt"
        
        if ckpt_path.exists():
            print(f"Fold {fold}: Loading existing checkpoint")
            model = MuQStatsModel.load_from_checkpoint(ckpt_path)
        else:
            train_ds = MERTDataset(MUQ_L9_12_DIR, LABELS, FOLD_ASSIGNMENTS, fold, "train", M1C_CONFIG["max_frames"])
            val_ds = MERTDataset(MUQ_L9_12_DIR, LABELS, FOLD_ASSIGNMENTS, fold, "val", M1C_CONFIG["max_frames"])
            
            print(f"Fold {fold}: Training ({len(train_ds)} train, {len(val_ds)} val)")
            
            train_dl = DataLoader(train_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=True,
                                  collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
            val_dl = DataLoader(val_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
                                collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
            
            model = make_muq_stats_model(M1C_CONFIG)
            callbacks = [
                ModelCheckpoint(dirpath=exp_checkpoint_dir, filename=f"fold{fold}_best",
                               monitor="val_r2", mode="max", save_top_k=1),
                EarlyStopping(monitor="val_r2", mode="max", patience=M1C_CONFIG["patience"], verbose=False),
            ]
            
            trainer = pl.Trainer(
                max_epochs=M1C_CONFIG["max_epochs"], callbacks=callbacks,
                logger=CSVLogger(save_dir=LOG_DIR, name=exp_id, version=f"fold{fold}"),
                accelerator="auto", devices=1, gradient_clip_val=M1C_CONFIG["gradient_clip_val"],
                enable_progress_bar=True, deterministic=True, log_every_n_steps=10,
            )
            
            trainer.fit(model, train_dl, val_dl)
            fold_results[fold] = float(callbacks[0].best_model_score or 0)
            print(f"Fold {fold} complete: val_r2 = {fold_results[fold]:.4f}")
            model = MuQStatsModel.load_from_checkpoint(ckpt_path)
        
        run_rclone(['rclone', 'copy', str(exp_checkpoint_dir), f'{GDRIVE_RESULTS}/checkpoints/{exp_id}'],
                   f"Uploading fold {fold} checkpoint")
        
        # Evaluate
        val_ds = MERTDataset(MUQ_L9_12_DIR, LABELS, FOLD_ASSIGNMENTS, fold, "val", M1C_CONFIG["max_frames"])
        val_dl = DataLoader(val_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
                            collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
        
        model.eval().to("cuda")
        with torch.no_grad():
            for batch in val_dl:
                pred = model(batch["embeddings"].cuda(), batch["attention_mask"].cuda(), batch.get("lengths"))
                all_preds.append(pred.cpu().numpy())
                all_labels.append(batch["labels"].numpy())
        
        if fold not in fold_results:
            fold_results[fold] = float(r2_score(np.vstack(all_labels[-len(val_ds):]), np.vstack(all_preds[-len(val_ds):])))
        
        # Save intermediate
        with open(existing_results_file, 'w') as f:
            json.dump({'experiment_id': exp_id, 'status': 'in_progress', 'fold_results': fold_results,
                      'audio_source': 'salamander', 'fold_method': 'piece_based'}, f, indent=2, default=numpy_serializer)
        run_rclone(['rclone', 'copyto', str(existing_results_file), f'{GDRIVE_RESULTS}/{exp_id}.json'],
                   f"Uploading intermediate results (fold {fold})")
        
        del model
        torch.cuda.empty_cache()
    
    overall_r2 = r2_score(np.vstack(all_labels), np.vstack(all_preds))
    A1A_R2 = overall_r2
    
    print(f"\n{exp_id} Overall R2: {overall_r2:.4f}")
    print(f"Per-fold R2: {[f'{fold_results[f]:.4f}' for f in range(4)]}")
    print(f"Fold std: {np.std([fold_results[f] for f in range(4)]):.4f}")
    
    final_results = {'experiment_id': exp_id, 'status': 'complete', 'overall_r2': overall_r2,
                    'fold_results': fold_results, 'fold_std': float(np.std([fold_results[f] for f in range(4)])),
                    'audio_source': 'salamander', 'fold_method': 'piece_based', 'n_samples': len(ALL_KEYS)}
    with open(existing_results_file, 'w') as f:
        json.dump(final_results, f, indent=2, default=numpy_serializer)
    
    sync_experiment_to_gdrive(exp_id, final_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS, ALL_RESULTS)
    ALL_RESULTS[exp_id] = final_results
else:
    results_file = RESULTS_DIR / f'{exp_id}.json'
    if results_file.exists():
        with open(results_file) as f:
            ALL_RESULTS[exp_id] = json.load(f)
        A1A_R2 = ALL_RESULTS[exp_id].get('overall_r2', 0)
    else:
        A1A_R2 = COMPLETED_CACHE.get(exp_id, 0)
    print(f"\nSKIP {exp_id}: already complete (R2={A1A_R2:.4f})")

In [ ]:
# A1b: Create performer-based fold assignments
from sklearn.model_selection import KFold

def get_performer_id(key: str) -> str:
    """Extract performer ID from sample key.
    
    Key format: 'Performer_Piece_Segment' or similar.
    We want to group by performer so same performer is never in both train and test.
    """
    parts = key.split('_')
    # First part is typically the performer ID (e.g., 'P01', 'performer1', etc.)
    return parts[-1] if parts else key

# Get unique performers and their samples
performers = defaultdict(list)
for key in ALL_KEYS:
    performer_id = get_performer_id(key)
    performers[performer_id].append(key)

print(f"Found {len(performers)} unique performers")
print(f"Samples per performer: min={min(len(v) for v in performers.values())}, "
      f"max={max(len(v) for v in performers.values())}, "
      f"mean={np.mean([len(v) for v in performers.values()]):.1f}")

# Create performer-based 4-fold split
performer_list = list(performers.keys())
kf = KFold(n_splits=4, shuffle=True, random_state=42)

PERFORMER_FOLD_ASSIGNMENTS = {}
PERFORMER_FOLD_BY_KEY = {}

for fold_idx, (train_perf_idx, val_perf_idx) in enumerate(kf.split(performer_list)):
    val_performers = [performer_list[i] for i in val_perf_idx]
    fold_keys = []
    for perf in val_performers:
        fold_keys.extend(performers[perf])
    PERFORMER_FOLD_ASSIGNMENTS[f"fold_{fold_idx}"] = fold_keys
    for key in fold_keys:
        PERFORMER_FOLD_BY_KEY[key] = fold_idx

print(f"\nPerformer-based fold sizes: {[len(PERFORMER_FOLD_ASSIGNMENTS[f'fold_{i}']) for i in range(4)]}")

# Save performer fold assignments
performer_folds_file = RESULTS_DIR / 'performer_fold_assignments.json'
with open(performer_folds_file, 'w') as f:
    json.dump(PERFORMER_FOLD_ASSIGNMENTS, f, indent=2)
run_rclone(['rclone', 'copyto', str(performer_folds_file), 
           f'{GDRIVE_RESULTS}/performer_fold_assignments.json'],
           "Uploading performer fold assignments")

In [ ]:
# A1b: Performer-based fold baseline
exp_id = 'A1b_performer_fold'

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    exp_checkpoint_dir = CHECKPOINT_ROOT / exp_id
    exp_checkpoint_dir.mkdir(parents=True, exist_ok=True)
    
    pl.seed_everything(42, workers=True)
    
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: {exp_id}")
    print(f"Description: MuQ L9-12 with PERFORMER-based folds")
    print(f"Audio: Salamander | Folds: Performer-based | Samples: {len(ALL_KEYS)}")
    print(f"{'='*70}")
    
    existing_folds = [f for f in range(4) if (exp_checkpoint_dir / f"fold{f}_best.ckpt").exists()]
    if existing_folds:
        print(f"Found existing checkpoints for folds: {existing_folds}")
    
    existing_results_file = RESULTS_DIR / f'{exp_id}.json'
    fold_results = {}
    if existing_results_file.exists():
        with open(existing_results_file) as f:
            existing_data = json.load(f)
            if 'fold_results' in existing_data:
                fold_results = {int(k): v for k, v in existing_data['fold_results'].items()}
    
    all_preds, all_labels = [], []
    
    for fold in range(4):
        ckpt_path = exp_checkpoint_dir / f"fold{fold}_best.ckpt"
        
        if ckpt_path.exists():
            print(f"Fold {fold}: Loading existing checkpoint")
            model = MuQStatsModel.load_from_checkpoint(ckpt_path)
        else:
            train_ds = MERTDataset(MUQ_L9_12_DIR, LABELS, PERFORMER_FOLD_ASSIGNMENTS, fold, "train", M1C_CONFIG["max_frames"])
            val_ds = MERTDataset(MUQ_L9_12_DIR, LABELS, PERFORMER_FOLD_ASSIGNMENTS, fold, "val", M1C_CONFIG["max_frames"])
            
            print(f"Fold {fold}: Training ({len(train_ds)} train, {len(val_ds)} val)")
            
            train_dl = DataLoader(train_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=True,
                                  collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
            val_dl = DataLoader(val_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
                                collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
            
            model = make_muq_stats_model(M1C_CONFIG)
            callbacks = [
                ModelCheckpoint(dirpath=exp_checkpoint_dir, filename=f"fold{fold}_best",
                               monitor="val_r2", mode="max", save_top_k=1),
                EarlyStopping(monitor="val_r2", mode="max", patience=M1C_CONFIG["patience"], verbose=False),
            ]
            
            trainer = pl.Trainer(
                max_epochs=M1C_CONFIG["max_epochs"], callbacks=callbacks,
                logger=CSVLogger(save_dir=LOG_DIR, name=exp_id, version=f"fold{fold}"),
                accelerator="auto", devices=1, gradient_clip_val=M1C_CONFIG["gradient_clip_val"],
                enable_progress_bar=True, deterministic=True, log_every_n_steps=10,
            )
            
            trainer.fit(model, train_dl, val_dl)
            fold_results[fold] = float(callbacks[0].best_model_score or 0)
            print(f"Fold {fold} complete: val_r2 = {fold_results[fold]:.4f}")
            model = MuQStatsModel.load_from_checkpoint(ckpt_path)
        
        run_rclone(['rclone', 'copy', str(exp_checkpoint_dir), f'{GDRIVE_RESULTS}/checkpoints/{exp_id}'],
                   f"Uploading fold {fold} checkpoint")
        
        val_ds = MERTDataset(MUQ_L9_12_DIR, LABELS, PERFORMER_FOLD_ASSIGNMENTS, fold, "val", M1C_CONFIG["max_frames"])
        val_dl = DataLoader(val_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
                            collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
        
        model.eval().to("cuda")
        with torch.no_grad():
            for batch in val_dl:
                pred = model(batch["embeddings"].cuda(), batch["attention_mask"].cuda(), batch.get("lengths"))
                all_preds.append(pred.cpu().numpy())
                all_labels.append(batch["labels"].numpy())
        
        if fold not in fold_results:
            fold_results[fold] = float(r2_score(np.vstack(all_labels[-len(val_ds):]), np.vstack(all_preds[-len(val_ds):])))
        
        with open(existing_results_file, 'w') as f:
            json.dump({'experiment_id': exp_id, 'status': 'in_progress', 'fold_results': fold_results,
                      'audio_source': 'salamander', 'fold_method': 'performer_based'}, f, indent=2, default=numpy_serializer)
        run_rclone(['rclone', 'copyto', str(existing_results_file), f'{GDRIVE_RESULTS}/{exp_id}.json'],
                   f"Uploading intermediate results (fold {fold})")
        
        del model
        torch.cuda.empty_cache()
    
    overall_r2 = r2_score(np.vstack(all_labels), np.vstack(all_preds))
    A1B_R2 = overall_r2
    
    print(f"\n{exp_id} Overall R2: {overall_r2:.4f}")
    print(f"Per-fold R2: {[f'{fold_results[f]:.4f}' for f in range(4)]}")
    print(f"Fold std: {np.std([fold_results[f] for f in range(4)]):.4f}")
    
    final_results = {'experiment_id': exp_id, 'status': 'complete', 'overall_r2': overall_r2,
                    'fold_results': fold_results, 'fold_std': float(np.std([fold_results[f] for f in range(4)])),
                    'audio_source': 'salamander', 'fold_method': 'performer_based', 'n_samples': len(ALL_KEYS)}
    with open(existing_results_file, 'w') as f:
        json.dump(final_results, f, indent=2, default=numpy_serializer)
    
    sync_experiment_to_gdrive(exp_id, final_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS, ALL_RESULTS)
    ALL_RESULTS[exp_id] = final_results
else:
    results_file = RESULTS_DIR / f'{exp_id}.json'
    if results_file.exists():
        with open(results_file) as f:
            ALL_RESULTS[exp_id] = json.load(f)
        A1B_R2 = ALL_RESULTS[exp_id].get('overall_r2', 0)
    else:
        A1B_R2 = COMPLETED_CACHE.get(exp_id, 0)
    print(f"\nSKIP {exp_id}: already complete (R2={A1B_R2:.4f})")

In [ ]:
# Cell 14: Analyze current fold distribution
def get_composer(key: str) -> str:
    """Extract composer from key (first part before underscore)."""
    return key.split('_')[0]

def get_composition_name(key: str) -> str:
    """Extract composition group name (without performer ID)."""
    parts = key.split("_")
    prefix = "_".join(parts[:-2])  # Everything except last 2 parts
    suffix = "_".join(parts[-1:])  # Last part (segment ID)
    return prefix + "_" + suffix

# Analyze current distribution
print("="*60)
print("CURRENT FOLD DISTRIBUTION ANALYSIS")
print("="*60)

for fold_id in range(4):
    fold_keys = FOLD_ASSIGNMENTS.get(f'fold_{fold_id}', [])
    
    # Composer distribution
    composers = defaultdict(int)
    for key in fold_keys:
        composers[get_composer(key)] += 1
    
    # Unique compositions
    compositions = set(get_composition_name(key) for key in fold_keys)
    
    print(f"\nFold {fold_id}: {len(fold_keys)} samples, {len(compositions)} compositions")
    print(f"  Composers: {dict(composers)}")

In [ ]:
# Cell 15: Implement stratified fold assignment function
import random
from sklearn.model_selection import StratifiedKFold

def create_stratified_folds(labels: Dict, n_folds: int = 4, seed: int = 42) -> Dict:
    """Create stratified fold assignments balancing composer and difficulty.
    
    Strategy:
    1. Group samples by composition (piece+segment without performer)
    2. Compute average label per composition as difficulty proxy
    3. Bin compositions into difficulty quartiles
    4. Stratify by (composer, difficulty_bin) to balance folds
    
    Returns:
        Dict with fold_0, fold_1, fold_2, fold_3 keys
    """
    random.seed(seed)
    np.random.seed(seed)
    
    all_keys = list(labels.keys())
    
    # Group by composition
    composition_groups = defaultdict(list)
    for key in all_keys:
        comp_name = get_composition_name(key)
        composition_groups[comp_name].append(key)
    
    # Compute composition-level features for stratification
    comp_features = []
    comp_names = list(composition_groups.keys())
    
    for comp_name in comp_names:
        keys = composition_groups[comp_name]
        composer = get_composer(keys[0])
        
        # Average label across all dimensions as difficulty proxy
        avg_labels = []
        for key in keys:
            avg_labels.append(np.mean(labels[key]))
        avg_difficulty = np.mean(avg_labels)
        
        comp_features.append({
            'comp_name': comp_name,
            'composer': composer,
            'avg_difficulty': avg_difficulty,
            'n_samples': len(keys),
        })
    
    # Bin difficulty into quartiles
    difficulties = [f['avg_difficulty'] for f in comp_features]
    quartiles = np.percentile(difficulties, [25, 50, 75])
    
    def get_difficulty_bin(d):
        if d < quartiles[0]:
            return 'Q1'
        elif d < quartiles[1]:
            return 'Q2'
        elif d < quartiles[2]:
            return 'Q3'
        return 'Q4'
    
    # Create stratification labels: (composer, difficulty_bin)
    strat_labels = []
    for f in comp_features:
        diff_bin = get_difficulty_bin(f['avg_difficulty'])
        strat_labels.append(f"{f['composer']}_{diff_bin}")
    
    # Use StratifiedKFold on compositions
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    
    fold_assignments = {f'fold_{i}': [] for i in range(n_folds)}
    
    for fold_idx, (_, val_indices) in enumerate(skf.split(comp_names, strat_labels)):
        for idx in val_indices:
            comp_name = comp_names[idx]
            fold_assignments[f'fold_{fold_idx}'].extend(composition_groups[comp_name])
    
    return fold_assignments

print("Stratified fold assignment function defined.")

In [ ]:
# Cell 16: Generate new stratified fold assignments
STRATIFIED_FOLD_ASSIGNMENTS = create_stratified_folds(LABELS, n_folds=4, seed=42)

print("="*60)
print("NEW STRATIFIED FOLD DISTRIBUTION")
print("="*60)

for fold_id in range(4):
    fold_keys = STRATIFIED_FOLD_ASSIGNMENTS.get(f'fold_{fold_id}', [])
    
    # Composer distribution
    composers = defaultdict(int)
    for key in fold_keys:
        composers[get_composer(key)] += 1
    
    # Unique compositions
    compositions = set(get_composition_name(key) for key in fold_keys)
    
    print(f"\nFold {fold_id}: {len(fold_keys)} samples, {len(compositions)} compositions")
    print(f"  Composers: {dict(composers)}")

# Compare sample counts
print("\n" + "="*60)
print("COMPARISON: Sample counts per fold")
print("="*60)
print(f"Original:   {[len(FOLD_ASSIGNMENTS.get(f'fold_{i}', [])) for i in range(4)]}")
print(f"Stratified: {[len(STRATIFIED_FOLD_ASSIGNMENTS.get(f'fold_{i}', [])) for i in range(4)]}")

In [ ]:
# A1c: Stratified fold baseline (balanced by composer/difficulty)
exp_id = 'A1c_stratified_fold'

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    exp_checkpoint_dir = CHECKPOINT_ROOT / exp_id
    exp_checkpoint_dir.mkdir(parents=True, exist_ok=True)
    
    pl.seed_everything(42, workers=True)
    
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: {exp_id}")
    print(f"Description: MuQ L9-12 with STRATIFIED folds (composer/difficulty balanced)")
    print(f"Audio: Salamander | Folds: Stratified | Samples: {len(ALL_KEYS)}")
    print(f"{'='*70}")
    
    existing_folds = [f for f in range(4) if (exp_checkpoint_dir / f"fold{f}_best.ckpt").exists()]
    if existing_folds:
        print(f"Found existing checkpoints for folds: {existing_folds}")
    
    existing_results_file = RESULTS_DIR / f'{exp_id}.json'
    fold_results = {}
    if existing_results_file.exists():
        with open(existing_results_file) as f:
            existing_data = json.load(f)
            if 'fold_results' in existing_data:
                fold_results = {int(k): v for k, v in existing_data['fold_results'].items()}
    
    all_preds, all_labels = [], []
    
    for fold in range(4):
        ckpt_path = exp_checkpoint_dir / f"fold{fold}_best.ckpt"
        
        if ckpt_path.exists():
            print(f"Fold {fold}: Loading existing checkpoint")
            model = MuQStatsModel.load_from_checkpoint(ckpt_path)
        else:
            train_ds = MERTDataset(MUQ_L9_12_DIR, LABELS, STRATIFIED_FOLD_ASSIGNMENTS, fold, "train", M1C_CONFIG["max_frames"])
            val_ds = MERTDataset(MUQ_L9_12_DIR, LABELS, STRATIFIED_FOLD_ASSIGNMENTS, fold, "val", M1C_CONFIG["max_frames"])
            
            print(f"Fold {fold}: Training ({len(train_ds)} train, {len(val_ds)} val)")
            
            train_dl = DataLoader(train_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=True,
                                  collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
            val_dl = DataLoader(val_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
                                collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
            
            model = make_muq_stats_model(M1C_CONFIG)
            callbacks = [
                ModelCheckpoint(dirpath=exp_checkpoint_dir, filename=f"fold{fold}_best",
                               monitor="val_r2", mode="max", save_top_k=1),
                EarlyStopping(monitor="val_r2", mode="max", patience=M1C_CONFIG["patience"], verbose=False),
            ]
            
            trainer = pl.Trainer(
                max_epochs=M1C_CONFIG["max_epochs"], callbacks=callbacks,
                logger=CSVLogger(save_dir=LOG_DIR, name=exp_id, version=f"fold{fold}"),
                accelerator="auto", devices=1, gradient_clip_val=M1C_CONFIG["gradient_clip_val"],
                enable_progress_bar=True, deterministic=True, log_every_n_steps=10,
            )
            
            trainer.fit(model, train_dl, val_dl)
            fold_results[fold] = float(callbacks[0].best_model_score or 0)
            print(f"Fold {fold} complete: val_r2 = {fold_results[fold]:.4f}")
            model = MuQStatsModel.load_from_checkpoint(ckpt_path)
        
        run_rclone(['rclone', 'copy', str(exp_checkpoint_dir), f'{GDRIVE_RESULTS}/checkpoints/{exp_id}'],
                   f"Uploading fold {fold} checkpoint")
        
        val_ds = MERTDataset(MUQ_L9_12_DIR, LABELS, STRATIFIED_FOLD_ASSIGNMENTS, fold, "val", M1C_CONFIG["max_frames"])
        val_dl = DataLoader(val_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
                            collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
        
        model.eval().to("cuda")
        with torch.no_grad():
            for batch in val_dl:
                pred = model(batch["embeddings"].cuda(), batch["attention_mask"].cuda(), batch.get("lengths"))
                all_preds.append(pred.cpu().numpy())
                all_labels.append(batch["labels"].numpy())
        
        if fold not in fold_results:
            fold_results[fold] = float(r2_score(np.vstack(all_labels[-len(val_ds):]), np.vstack(all_preds[-len(val_ds):])))
        
        with open(existing_results_file, 'w') as f:
            json.dump({'experiment_id': exp_id, 'status': 'in_progress', 'fold_results': fold_results,
                      'audio_source': 'salamander', 'fold_method': 'stratified'}, f, indent=2, default=numpy_serializer)
        run_rclone(['rclone', 'copyto', str(existing_results_file), f'{GDRIVE_RESULTS}/{exp_id}.json'],
                   f"Uploading intermediate results (fold {fold})")
        
        del model
        torch.cuda.empty_cache()
    
    overall_r2 = r2_score(np.vstack(all_labels), np.vstack(all_preds))
    A1C_R2 = overall_r2
    A1C_STD = np.std([fold_results[f] for f in range(4)])
    
    print(f"\n{exp_id} Overall R2: {overall_r2:.4f}")
    print(f"Per-fold R2: {[f'{fold_results[f]:.4f}' for f in range(4)]}")
    print(f"Fold std: {A1C_STD:.4f}")
    
    final_results = {'experiment_id': exp_id, 'status': 'complete', 'overall_r2': overall_r2,
                    'fold_results': fold_results, 'fold_std': float(A1C_STD),
                    'audio_source': 'salamander', 'fold_method': 'stratified', 'n_samples': len(ALL_KEYS)}
    with open(existing_results_file, 'w') as f:
        json.dump(final_results, f, indent=2, default=numpy_serializer)
    
    sync_experiment_to_gdrive(exp_id, final_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS, ALL_RESULTS)
    ALL_RESULTS[exp_id] = final_results
else:
    results_file = RESULTS_DIR / f'{exp_id}.json'
    if results_file.exists():
        with open(results_file) as f:
            ALL_RESULTS[exp_id] = json.load(f)
        A1C_R2 = ALL_RESULTS[exp_id].get('overall_r2', 0)
        A1C_STD = ALL_RESULTS[exp_id].get('fold_std', 0)
    else:
        A1C_R2 = COMPLETED_CACHE.get(exp_id, 0)
        A1C_STD = 0
    print(f"\nSKIP {exp_id}: already complete (R2={A1C_R2:.4f})")

In [ ]:
# A1 Results Comparison: Three-way fold method comparison
print("\n" + "="*70)
print("A1 COMPARISON: FOLD METHOD")
print("="*70)

# Get std values
A1A_STD = ALL_RESULTS.get('A1a_piece_fold', {}).get('fold_std', 0) if 'A1a_piece_fold' in ALL_RESULTS else 0
A1B_STD = ALL_RESULTS.get('A1b_performer_fold', {}).get('fold_std', 0) if 'A1b_performer_fold' in ALL_RESULTS else 0

print(f"\n{'Fold Method':<20} {'R2':>10} {'Fold Std':>12} {'Notes':<30}")
print("-"*75)
print(f"{'Piece-based':<20} {A1A_R2:>10.4f} {A1A_STD:>12.4f} {'Current approach':<30}")
print(f"{'Performer-based':<20} {A1B_R2:>10.4f} {A1B_STD:>12.4f} {'Tests unseen performer gen.':<30}")
print(f"{'Stratified':<20} {A1C_R2:>10.4f} {A1C_STD:>12.4f} {'Balanced composer/difficulty':<30}")

# Determine best fold method
# Criteria: Highest R2, but if performer-based is close and has lower std, prefer it for rigor
fold_methods = {
    'piece_based': {'r2': A1A_R2, 'std': A1A_STD, 'assignments': FOLD_ASSIGNMENTS},
    'performer_based': {'r2': A1B_R2, 'std': A1B_STD, 'assignments': PERFORMER_FOLD_ASSIGNMENTS},
    'stratified': {'r2': A1C_R2, 'std': A1C_STD, 'assignments': STRATIFIED_FOLD_ASSIGNMENTS},
}

# Pick method with highest R2
best_method = max(fold_methods.keys(), key=lambda k: fold_methods[k]['r2'])
BEST_A1_R2 = fold_methods[best_method]['r2']
BEST_FOLD_METHOD = best_method
BEST_FOLD_ASSIGNMENTS = fold_methods[best_method]['assignments']

# But add commentary
print(f"\nBest R2: {best_method} ({BEST_A1_R2:.4f})")

if A1B_R2 < A1A_R2:
    drop_pct = ((A1A_R2 - A1B_R2) / A1A_R2) * 100
    print(f"Performer-based shows {drop_pct:.1f}% lower R2 than piece-based.")
    print("This indicates some degree of performer-specific learning in piece-based folds.")

if A1C_STD < A1A_STD:
    print(f"Stratified folds reduce variance (std: {A1C_STD:.4f} vs {A1A_STD:.4f})")

print(f"\n>>> DECISION: Use {BEST_FOLD_METHOD.upper()} folds for A2 and Phase B+ <<<")
print("="*70)

### A2: Audio Source Comparison - Salamander vs Pianoteq Ensemble

**Hypothesis**: Training on multiple soundfonts (same MIDI, different audio renders) improves robustness by forcing the model to learn performance characteristics rather than soundfont-specific timbral features.

- **A2_salamander_baseline**: Uses A1 results (already computed)
- **A2_pianoteq_ensemble**: Train on ALL 6 Pianoteq soundfonts (7212 samples)

The Pianoteq ensemble effectively provides 6x data augmentation through timbre variation while keeping the performance labels constant.

In [ ]:
# A2a: Set up Pianoteq ensemble data (all 6 soundfonts)
# Download audio and extract/cache MuQ embeddings for each soundfont

PIANOTEQ_MUQ_DIRS = {}

for soundfont in PIANOTEQ_SOUNDFONTS:
    print(f"{'='*60}")
    print(f"Setting up {soundfont}")
    print(f"{'='*60}")

    # Audio directory
    sf_audio_dir = PIANOTEQ_ROOT / soundfont
    sf_audio_dir.mkdir(parents=True, exist_ok=True)

    # Download audio if not present
    existing_files = list(sf_audio_dir.glob('*.wav'))
    if len(existing_files) < len(ALL_KEYS):
        print(f"Downloading {soundfont} audio...")
        run_rclone(['rclone', 'copy', f'{GDRIVE_PIANOTEQ}/{soundfont}', str(sf_audio_dir), '--progress'],
                   f"Downloading {soundfont}")
    else:
        print(f"Audio already present: {len(existing_files)} files")

    # MuQ embedding directory
    sf_muq_dir = MUQ_CACHE_ROOT / f'pianoteq_{soundfont}_L9-12'
    sf_muq_dir.mkdir(parents=True, exist_ok=True)
    PIANOTEQ_MUQ_DIRS[soundfont] = sf_muq_dir

    # Check for cached embeddings on GDrive
    gdrive_muq_path = f'gdrive:crescendai_data/audio_baseline/muq_embeddings/pianoteq_{soundfont}_L9-12'
    result = subprocess.run(['rclone', 'lsf', gdrive_muq_path], capture_output=True, text=True)

    if result.returncode == 0 and result.stdout.strip():
        cached_count = len(result.stdout.strip().split(''))
        if cached_count >= len(ALL_KEYS):
            print(f"Downloading cached MuQ embeddings ({cached_count} files)...")
            run_rclone(['rclone', 'copy', gdrive_muq_path, str(sf_muq_dir), '--progress'],
                       f"Downloading {soundfont} MuQ embeddings")
            continue

    # Extract embeddings if not cached
    existing_emb = list(sf_muq_dir.glob('*.pt'))
    if len(existing_emb) < len(ALL_KEYS):
        print(f"Extracting MuQ L9-12 embeddings for {soundfont}...")
        # MuQExtractor takes layer_start/layer_end in constructor (layer_end is exclusive)
        extractor = MuQExtractor(layer_start=9, layer_end=13, cache_dir=sf_muq_dir)

        audio_files = sorted(sf_audio_dir.glob('*.wav'))
        for audio_file in tqdm(audio_files, desc=f"Extracting {soundfont}"):
            emb_file = sf_muq_dir / f"{audio_file.stem}.pt"
            if emb_file.exists():
                continue

            # extract_from_file returns averaged embedding directly and caches it
            extractor.extract_from_file(audio_file)

        del extractor
        torch.cuda.empty_cache()

        # Upload to GDrive cache
        print(f"Uploading {soundfont} embeddings to GDrive cache...")
        run_rclone(['rclone', 'copy', str(sf_muq_dir), gdrive_muq_path],
                   f"Caching {soundfont} embeddings")
    else:
        print(f"Embeddings already extracted: {len(existing_emb)} files")

print(f"{'='*60}")
print(f"Pianoteq ensemble setup complete: {len(PIANOTEQ_SOUNDFONTS)} soundfonts")
print(f"Total samples available: {len(ALL_KEYS) * len(PIANOTEQ_SOUNDFONTS)}")
print(f"{'='*60}")

In [ ]:
# A2b: Multi-soundfont dataset for Pianoteq ensemble training
from torch.utils.data import Dataset

class MultiSoundfontMERTDataset(Dataset):
    """Dataset that loads embeddings from multiple soundfont directories.
    Each sample appears multiple times (once per soundfont) with the same label.
    This effectively augments the data through timbre variation.
    """
    
    def __init__(
        self,
        soundfont_dirs: Dict[str, Path],  # soundfont_name -> embedding_dir
        labels: Dict[str, List[float]],
        fold_assignments: Dict[str, List[str]],
        fold: int,
        split: str,  # "train" or "val"
        max_frames: int = 300,
    ):
        self.soundfont_dirs = soundfont_dirs
        self.labels = labels
        self.max_frames = max_frames
        
        # Get keys for this split
        if split == "train":
            split_keys = []
            for f in range(4):
                if f != fold:
                    split_keys.extend(fold_assignments.get(f"fold_{f}", []))
        else:
            split_keys = fold_assignments.get(f"fold_{fold}", [])
        
        # Filter to keys that have labels and exist in all soundfonts
        self.samples = []  # List of (soundfont, key) tuples
        soundfont_names = list(soundfont_dirs.keys())
        
        for key in split_keys:
            if key not in labels:
                continue
            # Check if embedding exists in all soundfonts
            all_exist = all(
                (soundfont_dirs[sf] / f"{key}.pt").exists()
                for sf in soundfont_names
            )
            if all_exist:
                # Add one sample per soundfont
                for sf in soundfont_names:
                    self.samples.append((sf, key))
        
        self.samples = sorted(self.samples)
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        soundfont, key = self.samples[idx]
        emb_path = self.soundfont_dirs[soundfont] / f"{key}.pt"
        
        emb = torch.load(emb_path, weights_only=True)
        
        # Truncate/pad to max_frames
        if emb.shape[0] > self.max_frames:
            emb = emb[:self.max_frames]
        
        label = torch.tensor(self.labels[key], dtype=torch.float32)
        
        return {
            "key": f"{soundfont}_{key}",
            "embeddings": emb,
            "labels": label,
            "length": emb.shape[0],
        }

print("MultiSoundfontMERTDataset defined")
print(f"Available soundfonts: {list(PIANOTEQ_MUQ_DIRS.keys())}")

In [ ]:
# A2c: Train on Pianoteq ensemble (all 6 soundfonts)
exp_id = 'A2_pianoteq_ensemble'

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    exp_checkpoint_dir = CHECKPOINT_ROOT / exp_id
    exp_checkpoint_dir.mkdir(parents=True, exist_ok=True)
    
    pl.seed_everything(42, workers=True)
    
    # Use the best fold method from A1
    fold_assignments_to_use = BEST_FOLD_ASSIGNMENTS
    
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: {exp_id}")
    print(f"Description: MuQ L9-12 trained on ALL 6 Pianoteq soundfonts")
    print(f"Audio: Pianoteq ensemble | Folds: {BEST_FOLD_METHOD} | Soundfonts: {len(PIANOTEQ_SOUNDFONTS)}")
    print(f"Total training samples per fold: ~{len(ALL_KEYS) * len(PIANOTEQ_SOUNDFONTS) * 3 // 4}")
    print(f"{'='*70}")
    
    existing_folds = [f for f in range(4) if (exp_checkpoint_dir / f"fold{f}_best.ckpt").exists()]
    if existing_folds:
        print(f"Found existing checkpoints for folds: {existing_folds}")
    
    existing_results_file = RESULTS_DIR / f'{exp_id}.json'
    fold_results = {}
    if existing_results_file.exists():
        with open(existing_results_file) as f:
            existing_data = json.load(f)
            if 'fold_results' in existing_data:
                fold_results = {int(k): v for k, v in existing_data['fold_results'].items()}
    
    all_preds, all_labels = [], []
    
    for fold in range(4):
        ckpt_path = exp_checkpoint_dir / f"fold{fold}_best.ckpt"
        
        if ckpt_path.exists():
            print(f"Fold {fold}: Loading existing checkpoint")
            model = MuQStatsModel.load_from_checkpoint(ckpt_path)
        else:
            # Create multi-soundfont datasets
            train_ds = MultiSoundfontMERTDataset(
                PIANOTEQ_MUQ_DIRS, LABELS, fold_assignments_to_use, fold, "train", M1C_CONFIG["max_frames"]
            )
            val_ds = MultiSoundfontMERTDataset(
                PIANOTEQ_MUQ_DIRS, LABELS, fold_assignments_to_use, fold, "val", M1C_CONFIG["max_frames"]
            )
            
            print(f"Fold {fold}: Training ({len(train_ds)} train, {len(val_ds)} val)")
            
            train_dl = DataLoader(
                train_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=True,
                collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True,
            )
            val_dl = DataLoader(
                val_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
                collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True,
            )
            
            model = make_muq_stats_model(M1C_CONFIG)
            
            callbacks = [
                ModelCheckpoint(
                    dirpath=exp_checkpoint_dir, filename=f"fold{fold}_best",
                    monitor="val_r2", mode="max", save_top_k=1,
                ),
                EarlyStopping(
                    monitor="val_r2", mode="max", patience=M1C_CONFIG["patience"], verbose=False
                ),
            ]
            
            trainer = pl.Trainer(
                max_epochs=M1C_CONFIG["max_epochs"],
                callbacks=callbacks,
                logger=CSVLogger(save_dir=LOG_DIR, name=exp_id, version=f"fold{fold}"),
                accelerator="auto", devices=1,
                gradient_clip_val=M1C_CONFIG["gradient_clip_val"],
                enable_progress_bar=True, deterministic=True, log_every_n_steps=10,
            )
            
            trainer.fit(model, train_dl, val_dl)
            fold_results[fold] = float(callbacks[0].best_model_score or 0)
            print(f"Fold {fold} complete: val_r2 = {fold_results[fold]:.4f}")
            
            model = MuQStatsModel.load_from_checkpoint(ckpt_path)
        
        # Sync checkpoint to GDrive after each fold
        run_rclone(['rclone', 'copy', str(exp_checkpoint_dir), 
                   f'{GDRIVE_RESULTS}/checkpoints/{exp_id}'],
                   f"Uploading fold {fold} checkpoint")
        
        # Evaluate on validation set
        val_ds = MultiSoundfontMERTDataset(
            PIANOTEQ_MUQ_DIRS, LABELS, fold_assignments_to_use, fold, "val", M1C_CONFIG["max_frames"]
        )
        val_dl = DataLoader(
            val_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
            collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True,
        )
        
        model.eval().to("cuda")
        with torch.no_grad():
            for batch in val_dl:
                pred = model(
                    batch["embeddings"].cuda(),
                    batch["attention_mask"].cuda(),
                    batch.get("lengths"),
                )
                all_preds.append(pred.cpu().numpy())
                all_labels.append(batch["labels"].numpy())
        
        if fold not in fold_results:
            fold_preds = np.vstack(all_preds[-len(val_ds):])
            fold_labels = np.vstack(all_labels[-len(val_ds):])
            fold_results[fold] = float(r2_score(fold_labels, fold_preds))
        
        # Save intermediate results
        intermediate_results = {
            'experiment_id': exp_id,
            'status': 'in_progress',
            'fold_results': fold_results,
            'audio_source': 'pianoteq_ensemble',
            'soundfonts': PIANOTEQ_SOUNDFONTS,
            'fold_method': BEST_FOLD_METHOD,
        }
        with open(existing_results_file, 'w') as f:
            json.dump(intermediate_results, f, indent=2, default=numpy_serializer)
        
        run_rclone(['rclone', 'copyto', str(existing_results_file), 
                   f'{GDRIVE_RESULTS}/{exp_id}.json'],
                   f"Uploading intermediate results (fold {fold})")
        
        del model
        torch.cuda.empty_cache()
    
    # Final results
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    overall_r2 = r2_score(all_labels, all_preds)
    
    A2_PIANOTEQ_R2 = overall_r2
    print(f"\n{exp_id} Overall R2: {overall_r2:.4f}")
    print(f"Per-fold R2: {[f'{fold_results[f]:.4f}' for f in range(4)]}")
    
    final_results = {
        'experiment_id': exp_id,
        'status': 'complete',
        'overall_r2': overall_r2,
        'fold_results': fold_results,
        'audio_source': 'pianoteq_ensemble',
        'soundfonts': PIANOTEQ_SOUNDFONTS,
        'n_soundfonts': len(PIANOTEQ_SOUNDFONTS),
        'fold_method': BEST_FOLD_METHOD,
        'total_samples': len(ALL_KEYS) * len(PIANOTEQ_SOUNDFONTS),
    }
    with open(existing_results_file, 'w') as f:
        json.dump(final_results, f, indent=2, default=numpy_serializer)
    
    sync_experiment_to_gdrive(exp_id, final_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS, ALL_RESULTS)
    ALL_RESULTS[exp_id] = final_results
else:
    results_file = RESULTS_DIR / f'{exp_id}.json'
    if results_file.exists():
        with open(results_file) as f:
            ALL_RESULTS[exp_id] = json.load(f)
        A2_PIANOTEQ_R2 = ALL_RESULTS[exp_id].get('overall_r2', 0)
    else:
        A2_PIANOTEQ_R2 = COMPLETED_CACHE.get(exp_id, 0)
    print(f"\nSKIP {exp_id}: already complete (R2={A2_PIANOTEQ_R2:.4f})")

In [ ]:
# A2 Results Comparison: Salamander vs Pianoteq Ensemble
# Note: A2_salamander_baseline is effectively BEST_A1_R2 (using the best fold method)

A2_SALAMANDER_R2 = BEST_A1_R2  # From A1 comparison

print("\n" + "="*70)
print("A2 COMPARISON: SALAMANDER vs PIANOTEQ ENSEMBLE")
print("="*70)

print(f"\n{'Audio Source':<30} {'Samples':>10} {'R2':>10} {'Delta':>10}")
print("-"*65)
print(f"{'Salamander (single)':<30} {len(ALL_KEYS):>10} {A2_SALAMANDER_R2:>10.4f} {'baseline':>10}")
print(f"{'Pianoteq ensemble (6x)':<30} {len(ALL_KEYS)*6:>10} {A2_PIANOTEQ_R2:>10.4f} {A2_PIANOTEQ_R2 - A2_SALAMANDER_R2:>+10.4f}")

if A2_PIANOTEQ_R2 > A2_SALAMANDER_R2:
    improvement = ((A2_PIANOTEQ_R2 - A2_SALAMANDER_R2) / A2_SALAMANDER_R2) * 100
    print(f"\nPianoteq ensemble shows {improvement:.1f}% improvement!")
    print("Multi-soundfont training improves robustness through timbre augmentation.")
    BEST_AUDIO_SOURCE = 'pianoteq_ensemble'
    BEST_A2_R2 = A2_PIANOTEQ_R2
else:
    print(f"\nSalamander baseline is comparable or better.")
    print("Single soundfont training is sufficient for this task.")
    BEST_AUDIO_SOURCE = 'salamander'
    BEST_A2_R2 = A2_SALAMANDER_R2

print(f"\n>>> DECISION: Use {BEST_AUDIO_SOURCE.upper()} for subsequent experiments <<<")
print("="*70)

In [ ]:
# Phase A Summary: Optimal Configuration for Phase B
print("\n" + "="*70)
print("PHASE A COMPLETE: OPTIMAL CONFIGURATION DETERMINED")
print("="*70)

print(f"\n  Fold Method: {BEST_FOLD_METHOD.upper()}")
print(f"  Audio Source: {BEST_AUDIO_SOURCE.upper()}")
print(f"  Best R2 achieved: {max(BEST_A1_R2, A2_PIANOTEQ_R2):.4f}")

print("\n" + "-"*70)
print("Phase B experiments will use this configuration:")
print("-"*70)

if BEST_AUDIO_SOURCE == 'pianoteq_ensemble':
    print("  - Training data: Pianoteq ensemble (all 6 soundfonts)")
    print(f"  - Total samples: {len(ALL_KEYS) * len(PIANOTEQ_SOUNDFONTS)}")
    PHASE_B_MUQ_DIR = PIANOTEQ_MUQ_DIRS  # Dictionary for multi-soundfont
    PHASE_B_DATASET_CLASS = MultiSoundfontMERTDataset
else:
    print("  - Training data: Salamander (single soundfont)")
    print(f"  - Total samples: {len(ALL_KEYS)}")
    PHASE_B_MUQ_DIR = MUQ_L9_12_DIR
    PHASE_B_DATASET_CLASS = MERTDataset

if BEST_FOLD_METHOD == 'performer_based':
    print("  - Fold method: Performer-based (stricter generalization test)")
    PHASE_B_FOLDS = PERFORMER_FOLD_ASSIGNMENTS
else:
    print("  - Fold method: Piece-based (standard approach)")
    PHASE_B_FOLDS = FOLD_ASSIGNMENTS

# Save Phase A configuration
phase_a_config = {
    'best_fold_method': BEST_FOLD_METHOD,
    'best_audio_source': BEST_AUDIO_SOURCE,
    'A1_piece_fold_r2': A1_PIECE_FOLD_R2,
    'A1_performer_fold_r2': A1_PERFORMER_FOLD_R2,
    'A2_salamander_r2': A2_SALAMANDER_R2,
    'A2_pianoteq_r2': A2_PIANOTEQ_R2,
}
config_file = RESULTS_DIR / 'phase_a_config.json'
with open(config_file, 'w') as f:
    json.dump(phase_a_config, f, indent=2)

run_rclone(['rclone', 'copyto', str(config_file), 
           f'{GDRIVE_RESULTS}/phase_a_config.json'],
           "Uploading Phase A configuration")

print("\n" + "="*70)
print("Ready for Phase B experiments!")
print("="*70)

---
## PHASE B: Core Validation Experiments

Using the optimal configuration from Phase A, we now run the core validation experiments:

- **B1: Multi-Seed Stability** - Validate R2 is stable across 5 random seeds
- **B2: Stratified Folds** - Test if stratifying by composer/difficulty improves variance
- **B3: D7 Completion** - Complete any missing experiment results

In [ ]:
# B1: Multi-Seed Stability Analysis
# Tests whether the best configuration is stable across different random seeds

exp_id = 'B1_multi_seed_stability'
STABILITY_SEEDS = [42, 123, 456, 789, 1337]

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    exp_checkpoint_dir = CHECKPOINT_ROOT / exp_id
    exp_checkpoint_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: {exp_id}")
    print(f"Description: Multi-seed stability using optimal Phase A config")
    print(f"Audio: {BEST_AUDIO_SOURCE} | Folds: {BEST_FOLD_METHOD}")
    print(f"Seeds: {STABILITY_SEEDS}")
    print(f"{'='*70}")
    
    # Load existing results if resuming
    existing_results_file = RESULTS_DIR / f'{exp_id}.json'
    seed_results = {}
    if existing_results_file.exists():
        with open(existing_results_file) as f:
            existing_data = json.load(f)
            if 'seed_results' in existing_data:
                seed_results = {int(k): v for k, v in existing_data['seed_results'].items()}
    
    for seed in STABILITY_SEEDS:
        if seed in seed_results and seed_results[seed].get('overall_r2') is not None:
            print(f"\nSKIP seed {seed}: already complete (R2={seed_results[seed]['overall_r2']:.4f})")
            continue
        
        print(f"\n{'='*60}")
        print(f"SEED {seed}")
        print(f"{'='*60}")
        
        pl.seed_everything(seed, workers=True)
        torch.manual_seed(seed)
        np.random.seed(seed)
        
        seed_dir = exp_checkpoint_dir / f"seed_{seed}"
        seed_dir.mkdir(parents=True, exist_ok=True)
        
        fold_r2_scores = {}
        all_preds, all_labels = [], []
        
        for fold in range(4):
            ckpt_path = seed_dir / f"fold{fold}_best.ckpt"
            
            if ckpt_path.exists():
                print(f"  Fold {fold}: Loading existing checkpoint")
                model = MuQStatsModel.load_from_checkpoint(ckpt_path)
            else:
                # Use optimal configuration from Phase A
                if BEST_AUDIO_SOURCE == 'pianoteq_ensemble':
                    train_ds = MultiSoundfontMERTDataset(
                        PIANOTEQ_MUQ_DIRS, LABELS, PHASE_B_FOLDS, fold, "train", M1C_CONFIG["max_frames"]
                    )
                    val_ds = MultiSoundfontMERTDataset(
                        PIANOTEQ_MUQ_DIRS, LABELS, PHASE_B_FOLDS, fold, "val", M1C_CONFIG["max_frames"]
                    )
                else:
                    train_ds = MERTDataset(
                        MUQ_L9_12_DIR, LABELS, PHASE_B_FOLDS, fold, "train", M1C_CONFIG["max_frames"]
                    )
                    val_ds = MERTDataset(
                        MUQ_L9_12_DIR, LABELS, PHASE_B_FOLDS, fold, "val", M1C_CONFIG["max_frames"]
                    )
                
                print(f"  Fold {fold}: Training ({len(train_ds)} train, {len(val_ds)} val)")
                
                train_dl = DataLoader(
                    train_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=True,
                    collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True,
                )
                val_dl = DataLoader(
                    val_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
                    collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True,
                )
                
                model = make_muq_stats_model(M1C_CONFIG)
                
                callbacks = [
                    ModelCheckpoint(
                        dirpath=seed_dir, filename=f"fold{fold}_best",
                        monitor="val_r2", mode="max", save_top_k=1,
                    ),
                    EarlyStopping(
                        monitor="val_r2", mode="max", patience=M1C_CONFIG["patience"], verbose=False
                    ),
                ]
                
                trainer = pl.Trainer(
                    max_epochs=M1C_CONFIG["max_epochs"],
                    callbacks=callbacks,
                    logger=CSVLogger(save_dir=LOG_DIR, name=f"{exp_id}_seed{seed}", version=f"fold{fold}"),
                    accelerator="auto", devices=1,
                    gradient_clip_val=M1C_CONFIG["gradient_clip_val"],
                    enable_progress_bar=True, deterministic=True, log_every_n_steps=10,
                )
                
                trainer.fit(model, train_dl, val_dl)
                fold_r2_scores[fold] = float(callbacks[0].best_model_score or 0)
                print(f"    Fold {fold} complete: val_r2 = {fold_r2_scores[fold]:.4f}")
                
                model = MuQStatsModel.load_from_checkpoint(ckpt_path)
            
            # Sync after each fold
            run_rclone(['rclone', 'copy', str(seed_dir), 
                       f'{GDRIVE_RESULTS}/checkpoints/{exp_id}/seed_{seed}'],
                       f"Uploading seed {seed} fold {fold} checkpoint")
            
            # Evaluate
            if BEST_AUDIO_SOURCE == 'pianoteq_ensemble':
                val_ds = MultiSoundfontMERTDataset(
                    PIANOTEQ_MUQ_DIRS, LABELS, PHASE_B_FOLDS, fold, "val", M1C_CONFIG["max_frames"]
                )
            else:
                val_ds = MERTDataset(
                    MUQ_L9_12_DIR, LABELS, PHASE_B_FOLDS, fold, "val", M1C_CONFIG["max_frames"]
                )
            val_dl = DataLoader(
                val_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
                collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True,
            )
            
            model.eval().to("cuda")
            with torch.no_grad():
                for batch in val_dl:
                    pred = model(
                        batch["embeddings"].cuda(),
                        batch["attention_mask"].cuda(),
                        batch.get("lengths"),
                    )
                    all_preds.append(pred.cpu().numpy())
                    all_labels.append(batch["labels"].numpy())
            
            del model
            torch.cuda.empty_cache()
        
        # Compute seed-level metrics
        all_preds_arr = np.vstack(all_preds)
        all_labels_arr = np.vstack(all_labels)
        seed_r2 = r2_score(all_labels_arr, all_preds_arr)
        
        seed_results[seed] = {
            'overall_r2': float(seed_r2),
            'fold_r2': fold_r2_scores if fold_r2_scores else None,
        }
        
        print(f"\nSeed {seed} Overall R2: {seed_r2:.4f}")
        
        # Save intermediate results
        intermediate_results = {
            'experiment_id': exp_id,
            'status': 'in_progress',
            'seed_results': seed_results,
            'audio_source': BEST_AUDIO_SOURCE,
            'fold_method': BEST_FOLD_METHOD,
        }
        with open(existing_results_file, 'w') as f:
            json.dump(intermediate_results, f, indent=2, default=numpy_serializer)
        
        run_rclone(['rclone', 'copyto', str(existing_results_file), 
                   f'{GDRIVE_RESULTS}/{exp_id}.json'],
                   f"Uploading intermediate results (seed {seed})")
    
    # Compute stability statistics
    r2_values = [seed_results[s]['overall_r2'] for s in STABILITY_SEEDS]
    mean_r2 = np.mean(r2_values)
    std_r2 = np.std(r2_values)
    
    print(f"\n{'='*70}")
    print("MULTI-SEED STABILITY RESULTS")
    print(f"{'='*70}")
    print(f"Mean R2: {mean_r2:.4f} +/- {std_r2:.4f}")
    print(f"Range: [{min(r2_values):.4f}, {max(r2_values):.4f}]")
    print(f"CV: {(std_r2/mean_r2)*100:.2f}%")
    
    final_results = {
        'experiment_id': exp_id,
        'status': 'complete',
        'seed_results': seed_results,
        'mean_r2': float(mean_r2),
        'std_r2': float(std_r2),
        'r2_range': [float(min(r2_values)), float(max(r2_values))],
        'cv_percent': float((std_r2/mean_r2)*100),
        'audio_source': BEST_AUDIO_SOURCE,
        'fold_method': BEST_FOLD_METHOD,
    }
    with open(existing_results_file, 'w') as f:
        json.dump(final_results, f, indent=2, default=numpy_serializer)
    
    sync_experiment_to_gdrive(exp_id, final_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS, ALL_RESULTS)
    ALL_RESULTS[exp_id] = final_results
else:
    print(f"\nSKIP {exp_id}: already complete")

In [ ]:
# B2: Cross-Soundfont Generalization (Leave-One-Out)
# Train on 5 soundfonts, test on held-out 6th - tests timbre invariance
exp_id = 'B2_cross_soundfont'

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    exp_checkpoint_dir = CHECKPOINT_ROOT / exp_id
    exp_checkpoint_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: {exp_id}")
    print(f"Description: Leave-one-out soundfont generalization")
    print(f"Train on 5 soundfonts, test on held-out 6th")
    print(f"{'='*70}")
    
    existing_results_file = RESULTS_DIR / f'{exp_id}.json'
    loo_results = {}
    
    if existing_results_file.exists():
        with open(existing_results_file) as f:
            existing_data = json.load(f)
            if 'loo_results' in existing_data:
                loo_results = existing_data['loo_results']
    
    for held_out_sf in PIANOTEQ_SOUNDFONTS:
        if held_out_sf in loo_results:
            print(f"\nSKIP {held_out_sf}: already complete (R2={loo_results[held_out_sf]['test_r2']:.4f})")
            continue
        
        print(f"\n--- Held-out soundfont: {held_out_sf} ---")
        
        # Train soundfonts (all except held-out)
        train_soundfonts = {sf: PIANOTEQ_MUQ_DIRS[sf] for sf in PIANOTEQ_SOUNDFONTS if sf != held_out_sf}
        test_soundfont_dir = PIANOTEQ_MUQ_DIRS[held_out_sf]
        
        pl.seed_everything(42, workers=True)
        sf_checkpoint_dir = exp_checkpoint_dir / held_out_sf
        sf_checkpoint_dir.mkdir(parents=True, exist_ok=True)
        
        # Train on 5 soundfonts, 4-fold CV
        fold_results = {}
        all_test_preds, all_test_labels = [], []
        
        for fold in range(4):
            ckpt_path = sf_checkpoint_dir / f"fold{fold}_best.ckpt"
            
            if ckpt_path.exists():
                print(f"  Fold {fold}: Loading existing checkpoint")
                model = MuQStatsModel.load_from_checkpoint(ckpt_path)
            else:
                train_ds = MultiSoundfontMERTDataset(train_soundfonts, LABELS, BEST_FOLD_ASSIGNMENTS, 
                                                     fold, "train", M1C_CONFIG["max_frames"])
                val_ds = MultiSoundfontMERTDataset(train_soundfonts, LABELS, BEST_FOLD_ASSIGNMENTS,
                                                    fold, "val", M1C_CONFIG["max_frames"])
                
                print(f"  Fold {fold}: Training on {len(train_soundfonts)} soundfonts ({len(train_ds)} samples)")
                
                train_dl = DataLoader(train_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=True,
                                      collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
                val_dl = DataLoader(val_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
                                    collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
                
                model = make_muq_stats_model(M1C_CONFIG)
                callbacks = [
                    ModelCheckpoint(dirpath=sf_checkpoint_dir, filename=f"fold{fold}_best",
                                   monitor="val_r2", mode="max", save_top_k=1),
                    EarlyStopping(monitor="val_r2", mode="max", patience=M1C_CONFIG["patience"], verbose=False),
                ]
                
                trainer = pl.Trainer(
                    max_epochs=M1C_CONFIG["max_epochs"], callbacks=callbacks,
                    logger=CSVLogger(save_dir=LOG_DIR, name=f"{exp_id}_{held_out_sf}", version=f"fold{fold}"),
                    accelerator="auto", devices=1, gradient_clip_val=M1C_CONFIG["gradient_clip_val"],
                    enable_progress_bar=True, deterministic=True, log_every_n_steps=10,
                )
                
                trainer.fit(model, train_dl, val_dl)
                fold_results[fold] = float(callbacks[0].best_model_score or 0)
                model = MuQStatsModel.load_from_checkpoint(ckpt_path)
            
            run_rclone(['rclone', 'copy', str(sf_checkpoint_dir), f'{GDRIVE_RESULTS}/checkpoints/{exp_id}/{held_out_sf}'],
                       f"Uploading {held_out_sf} fold {fold}")
            
            # Test on held-out soundfont
            test_ds = MERTDataset(test_soundfont_dir, LABELS, BEST_FOLD_ASSIGNMENTS, fold, "val", M1C_CONFIG["max_frames"])
            test_dl = DataLoader(test_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
                                 collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
            
            model.eval().to("cuda")
            with torch.no_grad():
                for batch in test_dl:
                    pred = model(batch["embeddings"].cuda(), batch["attention_mask"].cuda(), batch.get("lengths"))
                    all_test_preds.append(pred.cpu().numpy())
                    all_test_labels.append(batch["labels"].numpy())
            
            del model
            torch.cuda.empty_cache()
        
        # Compute held-out soundfont R2
        test_r2 = r2_score(np.vstack(all_test_labels), np.vstack(all_test_preds))
        
        loo_results[held_out_sf] = {
            'train_soundfonts': [sf for sf in PIANOTEQ_SOUNDFONTS if sf != held_out_sf],
            'fold_results': fold_results,
            'test_r2': float(test_r2),
        }
        
        print(f"  {held_out_sf} held-out test R2: {test_r2:.4f}")
        
        # Save intermediate
        with open(existing_results_file, 'w') as f:
            json.dump({'experiment_id': exp_id, 'status': 'in_progress', 'loo_results': loo_results}, 
                     f, indent=2, default=numpy_serializer)
        run_rclone(['rclone', 'copyto', str(existing_results_file), f'{GDRIVE_RESULTS}/{exp_id}.json'],
                   f"Uploading intermediate results ({held_out_sf})")
    
    # Summary
    test_r2s = [loo_results[sf]['test_r2'] for sf in PIANOTEQ_SOUNDFONTS]
    mean_test_r2 = np.mean(test_r2s)
    std_test_r2 = np.std(test_r2s)
    
    print(f"\n{'='*70}")
    print("CROSS-SOUNDFONT GENERALIZATION RESULTS")
    print(f"{'='*70}")
    print(f"\n{'Held-out Soundfont':<30} {'Test R2':>10}")
    print("-"*45)
    for sf in PIANOTEQ_SOUNDFONTS:
        print(f"{sf:<30} {loo_results[sf]['test_r2']:>10.4f}")
    print("-"*45)
    print(f"{'Mean'::<30} {mean_test_r2:>10.4f}")
    print(f"{'Std'::<30} {std_test_r2:>10.4f}")
    
    final_results = {
        'experiment_id': exp_id, 'status': 'complete',
        'loo_results': loo_results,
        'mean_test_r2': float(mean_test_r2),
        'std_test_r2': float(std_test_r2),
        'fold_method': BEST_FOLD_METHOD,
    }
    with open(existing_results_file, 'w') as f:
        json.dump(final_results, f, indent=2, default=numpy_serializer)
    
    sync_experiment_to_gdrive(exp_id, final_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS, ALL_RESULTS)
    ALL_RESULTS[exp_id] = final_results
else:
    print(f"\nSKIP {exp_id}: already complete")

### B3: Statistical Rigor - Bootstrap CIs and Significance Tests

**Goal**: Compute bootstrap confidence intervals and statistical significance for all Phase A comparisons.

This ensures our claims about fold methods and audio sources are statistically sound.

In [ ]:
# B3: Bootstrap CIs and Significance Tests
exp_id = 'B3_bootstrap_significance'

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: {exp_id}")
    print(f"Description: Bootstrap CIs and paired t-tests for all comparisons")
    print(f"{'='*70}")
    
    N_BOOTSTRAP = 1000
    
    def bootstrap_ci(values, n_bootstrap=N_BOOTSTRAP, ci=0.95):
        """Compute bootstrap confidence interval."""
        bootstrapped = []
        for _ in range(n_bootstrap):
            sample = np.random.choice(values, size=len(values), replace=True)
            bootstrapped.append(np.mean(sample))
        lower = np.percentile(bootstrapped, (1-ci)/2 * 100)
        upper = np.percentile(bootstrapped, (1+ci)/2 * 100)
        return lower, upper
    
    results = {'comparisons': {}}
    
    # A1: Fold method comparison
    print("\n--- A1: Fold Method CIs ---")
    fold_method_r2s = {
        'piece_based': A1A_R2,
        'performer_based': A1B_R2,
        'stratified': A1C_R2,
    }
    
    for method, r2 in fold_method_r2s.items():
        # Get per-fold R2s for bootstrap
        exp_results = ALL_RESULTS.get(f'A1{"a" if method=="piece_based" else "b" if method=="performer_based" else "c"}_{method.replace("_based", "") if "based" in method else method}_fold', {})
        fold_r2s = list(exp_results.get('fold_results', {}).values())
        if fold_r2s:
            ci_low, ci_high = bootstrap_ci(fold_r2s)
            print(f"  {method}: R2={r2:.4f} [95% CI: {ci_low:.4f}, {ci_high:.4f}]")
            results['comparisons'][f'A1_{method}'] = {
                'r2': r2, 'ci_95': [float(ci_low), float(ci_high)], 'fold_r2s': fold_r2s
            }
    
    # Paired t-test: piece vs performer
    if 'A1a_piece_fold' in ALL_RESULTS and 'A1b_performer_fold' in ALL_RESULTS:
        piece_folds = list(ALL_RESULTS['A1a_piece_fold'].get('fold_results', {}).values())
        perf_folds = list(ALL_RESULTS['A1b_performer_fold'].get('fold_results', {}).values())
        if len(piece_folds) == len(perf_folds) == 4:
            t_stat, p_value = stats.ttest_rel(piece_folds, perf_folds)
            print(f"\n  Paired t-test (piece vs performer): t={t_stat:.3f}, p={p_value:.4f}")
            results['comparisons']['piece_vs_performer'] = {'t_stat': float(t_stat), 'p_value': float(p_value)}
    
    # A2: Audio source comparison  
    print("\n--- A2: Audio Source CIs ---")
    if 'A2_pianoteq_ensemble' in ALL_RESULTS:
        ensemble_folds = list(ALL_RESULTS['A2_pianoteq_ensemble'].get('fold_results', {}).values())
        if ensemble_folds:
            ci_low, ci_high = bootstrap_ci(ensemble_folds)
            print(f"  Pianoteq ensemble: R2={A2_PIANOTEQ_R2:.4f} [95% CI: {ci_low:.4f}, {ci_high:.4f}]")
            results['comparisons']['A2_pianoteq'] = {
                'r2': A2_PIANOTEQ_R2, 'ci_95': [float(ci_low), float(ci_high)]
            }
    
    # B1: Multi-seed stability
    if 'B1_multi_seed_stability' in ALL_RESULTS:
        seed_r2s = [ALL_RESULTS['B1_multi_seed_stability']['seed_results'][str(s)]['overall_r2'] 
                    for s in [42, 123, 456, 789, 1337] 
                    if str(s) in ALL_RESULTS['B1_multi_seed_stability'].get('seed_results', {})]
        if seed_r2s:
            ci_low, ci_high = bootstrap_ci(seed_r2s)
            print(f"\n--- B1: Multi-Seed Stability ---")
            print(f"  Mean R2={np.mean(seed_r2s):.4f} +/- {np.std(seed_r2s):.4f}")
            print(f"  [95% CI: {ci_low:.4f}, {ci_high:.4f}]")
            results['comparisons']['B1_stability'] = {
                'mean_r2': float(np.mean(seed_r2s)), 'std_r2': float(np.std(seed_r2s)),
                'ci_95': [float(ci_low), float(ci_high)]
            }
    
    # B2: Cross-soundfont
    if 'B2_cross_soundfont' in ALL_RESULTS:
        loo_r2s = [ALL_RESULTS['B2_cross_soundfont']['loo_results'][sf]['test_r2'] 
                   for sf in PIANOTEQ_SOUNDFONTS 
                   if sf in ALL_RESULTS['B2_cross_soundfont'].get('loo_results', {})]
        if loo_r2s:
            ci_low, ci_high = bootstrap_ci(loo_r2s)
            print(f"\n--- B2: Cross-Soundfont Generalization ---")
            print(f"  Mean held-out R2={np.mean(loo_r2s):.4f} +/- {np.std(loo_r2s):.4f}")
            print(f"  [95% CI: {ci_low:.4f}, {ci_high:.4f}]")
            results['comparisons']['B2_cross_soundfont'] = {
                'mean_r2': float(np.mean(loo_r2s)), 'std_r2': float(np.std(loo_r2s)),
                'ci_95': [float(ci_low), float(ci_high)]
            }
    
    # Save results
    existing_results_file = RESULTS_DIR / f'{exp_id}.json'
    final_results = {
        'experiment_id': exp_id, 'status': 'complete',
        'n_bootstrap': N_BOOTSTRAP,
        'comparisons': results['comparisons'],
    }
    with open(existing_results_file, 'w') as f:
        json.dump(final_results, f, indent=2, default=numpy_serializer)
    
    sync_experiment_to_gdrive(exp_id, final_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS, ALL_RESULTS)
    ALL_RESULTS[exp_id] = final_results
    
    print(f"\n{'='*70}")
else:
    print(f"\nSKIP {exp_id}: already complete")

---
## PHASE C: Cross-Dataset Transfer

Test generalization to external datasets. Uses the optimal configuration from Phase A.

### Storage Requirements
- **MAESTRO v2.0.0**: ~300GB (115GB ZIP + ~200GB extracted)
- **ASAP**: ~1GB (metadata only, uses MAESTRO audio)
- **PSyllabus**: ~20-50GB (YouTube audio via yt-dlp)
- **Total**: ~320-350GB

### Setup Order
1. Run **MAESTRO + ASAP Setup** cell (downloads ~115GB, extracts ~200GB)
2. Run **PSyllabus Setup** cell (downloads ~550 YouTube videos)
3. Run experiments C1, C2, C3

### Experiments

**C1: PSyllabus Difficulty Correlation**
- Correlate model predictions with external piano difficulty ratings (1-11 Henle scale)
- Tests whether model captures skill-level-related features

**C2: ASAP Multi-Performer Analysis**
- Analyze prediction variance across different performers playing the same pieces
- High variance = model captures performer-specific interpretation
- Low variance = model captures piece-invariant features

**C3: MAESTRO Zero-Shot Transfer**
- Zero-shot evaluation on MAESTRO professional piano recordings
- No ground truth labels - analyze prediction distribution
- Expect higher mean predictions (professional competition recordings)


In [ ]:
# MAESTRO + ASAP Dataset Setup
# Downloads MAESTRO v2.0.0 (~115GB ZIP, ~200GB extracted) and clones ASAP repository
# ASAP links multiple performances of same pieces to MAESTRO audio
# Storage required: ~300GB

import zipfile
import urllib.request
from tqdm import tqdm

MAESTRO_VERSION = 'v2.0.0'
MAESTRO_URL = f'https://storage.googleapis.com/magentadata/datasets/maestro/{MAESTRO_VERSION}/maestro-{MAESTRO_VERSION}.zip'
MAESTRO_ROOT = DATA_ROOT / 'maestro'
MAESTRO_ZIP = MAESTRO_ROOT / f'maestro-{MAESTRO_VERSION}.zip'
MAESTRO_EXTRACTED = MAESTRO_ROOT / f'maestro-{MAESTRO_VERSION}'
MAESTRO_AUDIO_DIR = MAESTRO_EXTRACTED / '2018'  # MAESTRO organizes by year

ASAP_DIR = DATA_ROOT / 'asap'
ASAP_REPO = ASAP_DIR / 'asap-dataset'

MAESTRO_ROOT.mkdir(parents=True, exist_ok=True)
ASAP_DIR.mkdir(parents=True, exist_ok=True)

class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_with_progress(url: str, output_path: Path, desc: str):
    """Download file with progress bar."""
    with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc=desc) as t:
        urllib.request.urlretrieve(url, output_path, reporthook=t.update_to)

# Step 1: Download MAESTRO if not exists
if not MAESTRO_EXTRACTED.exists():
    if not MAESTRO_ZIP.exists():
        print(f"Downloading MAESTRO {MAESTRO_VERSION} (~115GB)...")
        print("This will take a while depending on network speed.")
        download_with_progress(MAESTRO_URL, MAESTRO_ZIP, f"MAESTRO {MAESTRO_VERSION}")
    else:
        print(f"MAESTRO ZIP exists: {MAESTRO_ZIP}")
    
    # Extract MAESTRO
    print(f"Extracting MAESTRO (~200GB uncompressed)...")
    with zipfile.ZipFile(MAESTRO_ZIP, 'r') as zf:
        zf.extractall(MAESTRO_ROOT)
    print(f"Extracted to: {MAESTRO_EXTRACTED}")
    
    # Optionally delete ZIP to save space
    # MAESTRO_ZIP.unlink()
else:
    print(f"MAESTRO exists: {MAESTRO_EXTRACTED}")

# Step 2: Clone ASAP repository
if not ASAP_REPO.exists():
    print("Cloning ASAP repository...")
    result = subprocess.run(
        ['git', 'clone', 'https://github.com/fosfrancesco/asap-dataset.git', str(ASAP_REPO)],
        capture_output=True, text=True
    )
    if result.returncode != 0:
        raise RuntimeError(f"Failed to clone ASAP: {result.stderr}")
    print(f"Cloned to: {ASAP_REPO}")
else:
    print(f"ASAP repo exists: {ASAP_REPO}")

# Step 3: Load MAESTRO metadata and create mapping
maestro_csv = MAESTRO_EXTRACTED / f'maestro-{MAESTRO_VERSION}.csv'
if maestro_csv.exists():
    import csv
    MAESTRO_METADATA = []
    with open(maestro_csv) as f:
        reader = csv.DictReader(f)
        for row in reader:
            MAESTRO_METADATA.append(row)
    print(f"Loaded {len(MAESTRO_METADATA)} MAESTRO entries")
    
    # Create canonical_title -> entries mapping
    MAESTRO_BY_TITLE = {}
    for entry in MAESTRO_METADATA:
        title = entry['canonical_title']
        if title not in MAESTRO_BY_TITLE:
            MAESTRO_BY_TITLE[title] = []
        MAESTRO_BY_TITLE[title].append(entry)
    
    # Find pieces with multiple performers (for ASAP-style analysis)
    MAESTRO_MULTI_PERFORMER = {k: v for k, v in MAESTRO_BY_TITLE.items() if len(v) >= 2}
    print(f"Pieces with 2+ performances: {len(MAESTRO_MULTI_PERFORMER)}")
else:
    print(f"MAESTRO CSV not found: {maestro_csv}")
    MAESTRO_METADATA = []
    MAESTRO_BY_TITLE = {}
    MAESTRO_MULTI_PERFORMER = {}

# Step 4: Load ASAP metadata and link to MAESTRO
asap_metadata_dir = ASAP_REPO / 'metadata'
if asap_metadata_dir.exists():
    import yaml
    
    ASAP_ENTRIES = []
    for yaml_file in asap_metadata_dir.glob('*.yml'):
        with open(yaml_file) as f:
            try:
                data = yaml.safe_load(f)
                if data:
                    data['_file'] = yaml_file.stem
                    ASAP_ENTRIES.append(data)
            except Exception:
                pass
    
    print(f"Loaded {len(ASAP_ENTRIES)} ASAP entries")
    
    # Find ASAP entries that link to MAESTRO
    ASAP_WITH_MAESTRO = [e for e in ASAP_ENTRIES if 'maestro_audio_path' in str(e)]
    print(f"ASAP entries with MAESTRO links: {len(ASAP_WITH_MAESTRO)}")
else:
    print("ASAP metadata directory not found")
    ASAP_ENTRIES = []
    ASAP_WITH_MAESTRO = []

print(f"\nMAESTRO + ASAP setup complete!")
print(f"  MAESTRO audio: {MAESTRO_EXTRACTED}")
print(f"  ASAP repo: {ASAP_REPO}")


In [ ]:
# PSyllabus Dataset Setup
# Downloads metadata from Zenodo and audio from YouTube via yt-dlp
# Storage required: ~20-50GB depending on sampling
# Note: YouTube downloads can be slow due to rate limiting

import time
import random
import requests

# Install yt-dlp if not available
try:
    import yt_dlp
except ImportError:
    print("Installing yt-dlp...")
    subprocess.run(['pip', 'install', '-q', 'yt-dlp'], check=True)
    import yt_dlp

PSYLLABUS_DIR = DATA_ROOT / 'psyllabus'
PSYLLABUS_METADATA_URL = 'https://zenodo.org/records/14794592/files/new_clean_data.json?download=1'
PSYLLABUS_METADATA_FILE = PSYLLABUS_DIR / 'new_clean_data.json'
PSYLLABUS_AUDIO_DIR = PSYLLABUS_DIR / 'audio'
PSYLLABUS_CHECKPOINT_FILE = PSYLLABUS_DIR / 'download_checkpoint.json'
PSYLLABUS_MUQ_DIR = PSYLLABUS_DIR / 'muq_cache'

PSYLLABUS_DIR.mkdir(parents=True, exist_ok=True)
PSYLLABUS_AUDIO_DIR.mkdir(parents=True, exist_ok=True)
PSYLLABUS_MUQ_DIR.mkdir(parents=True, exist_ok=True)

# Target samples per difficulty level (1-11)
SAMPLES_PER_DIFFICULTY = 50

def download_with_requests(url: str, dest: Path, desc: str):
    """Download file with progress bar using requests."""
    response = requests.get(url, stream=True)
    response.raise_for_status()
    total = int(response.headers.get('content-length', 0))
    
    with open(dest, 'wb') as f:
        with tqdm(total=total, unit='B', unit_scale=True, desc=desc) as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                pbar.update(len(chunk))

# Step 1: Download metadata from Zenodo
if not PSYLLABUS_METADATA_FILE.exists():
    print("Downloading PSyllabus metadata from Zenodo...")
    download_with_requests(PSYLLABUS_METADATA_URL, PSYLLABUS_METADATA_FILE, "PSyllabus metadata")
else:
    print(f"PSyllabus metadata exists: {PSYLLABUS_METADATA_FILE}")

# Step 2: Load and parse metadata
with open(PSYLLABUS_METADATA_FILE) as f:
    psyllabus_raw = json.load(f)

# Parse entries - structure may vary
PSYLLABUS_ENTRIES = []
if isinstance(psyllabus_raw, list):
    PSYLLABUS_ENTRIES = psyllabus_raw
elif isinstance(psyllabus_raw, dict):
    if 'data' in psyllabus_raw:
        PSYLLABUS_ENTRIES = psyllabus_raw['data']
    else:
        PSYLLABUS_ENTRIES = list(psyllabus_raw.values())

# Filter entries with difficulty and YouTube URL
PSYLLABUS_VALID = []
for entry in PSYLLABUS_ENTRIES:
    if isinstance(entry, dict):
        difficulty = entry.get('difficulty') or entry.get('henle_level') or entry.get('level')
        youtube_url = entry.get('youtube_url') or entry.get('url') or entry.get('video_url')
        if difficulty is not None and youtube_url:
            try:
                diff_int = int(difficulty)
                if 1 <= diff_int <= 11:
                    PSYLLABUS_VALID.append({
                        'id': entry.get('id', len(PSYLLABUS_VALID)),
                        'title': entry.get('title', entry.get('piece_name', 'Unknown')),
                        'composer': entry.get('composer', 'Unknown'),
                        'difficulty': diff_int,
                        'youtube_url': youtube_url,
                    })
            except (ValueError, TypeError):
                pass

print(f"Found {len(PSYLLABUS_VALID)} PSyllabus entries with difficulty and YouTube URL")

# Step 3: Sample evenly across difficulty levels
from collections import defaultdict
by_difficulty = defaultdict(list)
for entry in PSYLLABUS_VALID:
    by_difficulty[entry['difficulty']].append(entry)

print("\nEntries by difficulty level:")
for diff in sorted(by_difficulty.keys()):
    print(f"  Level {diff}: {len(by_difficulty[diff])} entries")

# Sample up to SAMPLES_PER_DIFFICULTY from each level
random.seed(42)  # Reproducible sampling
PSYLLABUS_SAMPLED = []
for diff in sorted(by_difficulty.keys()):
    entries = by_difficulty[diff]
    n_sample = min(len(entries), SAMPLES_PER_DIFFICULTY)
    sampled = random.sample(entries, n_sample)
    PSYLLABUS_SAMPLED.extend(sampled)

print(f"\nSampled {len(PSYLLABUS_SAMPLED)} entries for download")

# Step 4: Download YouTube audio with checkpointing
# Load checkpoint if exists
if PSYLLABUS_CHECKPOINT_FILE.exists():
    with open(PSYLLABUS_CHECKPOINT_FILE) as f:
        checkpoint = json.load(f)
    downloaded_ids = set(checkpoint.get('downloaded', []))
    failed_ids = set(checkpoint.get('failed', []))
else:
    downloaded_ids = set()
    failed_ids = set()

# Check already downloaded files
existing_files = {p.stem for p in PSYLLABUS_AUDIO_DIR.glob('*.wav')}
downloaded_ids.update(str(eid) for eid in existing_files if eid.isdigit())

to_download = [e for e in PSYLLABUS_SAMPLED 
               if str(e['id']) not in downloaded_ids and str(e['id']) not in failed_ids]

print(f"Already downloaded: {len(downloaded_ids)}")
print(f"Previously failed: {len(failed_ids)}")
print(f"Remaining to download: {len(to_download)}")

def save_checkpoint():
    with open(PSYLLABUS_CHECKPOINT_FILE, 'w') as f:
        json.dump({'downloaded': list(downloaded_ids), 'failed': list(failed_ids)}, f)

# Download with yt-dlp
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192',
    }],
    'quiet': True,
    'no_warnings': True,
}

for i, entry in enumerate(tqdm(to_download, desc="Downloading PSyllabus audio")):
    entry_id = str(entry['id'])
    output_path = PSYLLABUS_AUDIO_DIR / f"{entry_id}.wav"
    
    try:
        ydl_opts['outtmpl'] = str(PSYLLABUS_AUDIO_DIR / f"{entry_id}.%(ext)s")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([entry['youtube_url']])
        
        if output_path.exists():
            downloaded_ids.add(entry_id)
        else:
            failed_ids.add(entry_id)
    except Exception as e:
        failed_ids.add(entry_id)
        print(f"\nFailed to download {entry_id}: {e}")
    
    # Checkpoint every 10 downloads
    if (i + 1) % 10 == 0:
        save_checkpoint()
    
    # Rate limiting - wait 1-3 seconds between downloads
    time.sleep(random.uniform(1, 3))

save_checkpoint()

# Final status
final_downloaded = list(PSYLLABUS_AUDIO_DIR.glob('*.wav'))
print(f"\nPSyllabus setup complete!")
print(f"  Downloaded audio files: {len(final_downloaded)}")
print(f"  Failed downloads: {len(failed_ids)}")

# Create mapping of downloaded files with difficulty
PSYLLABUS_DOWNLOADED = []
for entry in PSYLLABUS_SAMPLED:
    audio_path = PSYLLABUS_AUDIO_DIR / f"{entry['id']}.wav"
    if audio_path.exists():
        entry['audio_path'] = str(audio_path)
        PSYLLABUS_DOWNLOADED.append(entry)

print(f"  Ready for analysis: {len(PSYLLABUS_DOWNLOADED)} samples")


In [ ]:
# C1: PSyllabus Difficulty Correlation
# Correlate PercePiano predictions with external difficulty ratings
# Requires: PSYLLABUS_DOWNLOADED from setup cell

exp_id = 'C1_psyllabus_difficulty'

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: {exp_id}")
    print(f"Description: Correlate predictions with PSyllabus difficulty ratings")
    print(f"{'='*70}")
    
    # Check if PSyllabus data is available
    if 'PSYLLABUS_DOWNLOADED' not in dir() or len(PSYLLABUS_DOWNLOADED) == 0:
        raise RuntimeError("Run PSyllabus setup cell first!")
    
    print(f"PSyllabus samples available: {len(PSYLLABUS_DOWNLOADED)}")
    
    # Step 1: Extract MuQ embeddings for PSyllabus audio
    psyllabus_keys = [str(e['id']) for e in PSYLLABUS_DOWNLOADED]
    cached_psyllabus = {p.stem for p in PSYLLABUS_MUQ_DIR.glob('*.pt')}
    missing_psyllabus = [k for k in psyllabus_keys if k not in cached_psyllabus]
    
    if missing_psyllabus:
        print(f"Extracting {len(missing_psyllabus)} MuQ embeddings...")
        extract_muq_embeddings(
            PSYLLABUS_AUDIO_DIR, PSYLLABUS_MUQ_DIR, missing_psyllabus,
            layer_start=9, layer_end=13  # MuQ L9-12
        )
    
    # Step 2: Load best model from Phase A/B
    best_ckpt = None
    # Try several possible checkpoint locations
    ckpt_candidates = [
        CHECKPOINT_ROOT / 'A1c_stratified_fold' / 'fold0_best.ckpt',
        CHECKPOINT_ROOT / 'A1a_piece_fold' / 'fold0_best.ckpt',
        CHECKPOINT_ROOT / 'B1_multi_seed_stability' / 'seed_42' / 'fold0_best.ckpt',
    ]
    for ckpt in ckpt_candidates:
        if ckpt.exists():
            best_ckpt = ckpt
            break
    
    if best_ckpt is None:
        raise RuntimeError("No trained model found! Run Phase A experiments first.")
    
    print(f"Loading model from {best_ckpt.name}...")
    model = MuQStatsModel.load_from_checkpoint(best_ckpt)
    model = model.to('cuda').eval()
    
    # Step 3: Run inference on PSyllabus
    predictions = []
    difficulties = []
    
    for entry in tqdm(PSYLLABUS_DOWNLOADED, desc="PSyllabus inference"):
        emb_path = PSYLLABUS_MUQ_DIR / f"{entry['id']}.pt"
        if not emb_path.exists():
            continue
        
        with torch.no_grad():
            emb = torch.load(emb_path, weights_only=True).unsqueeze(0).cuda()
            if emb.shape[1] > 300:  # max_frames
                emb = emb[:, :300, :]
            mask = torch.ones(1, emb.shape[1], dtype=torch.bool).cuda()
            pred = model(emb, mask).cpu().numpy()[0]
            predictions.append(pred)
            difficulties.append(entry['difficulty'])
    
    del model
    torch.cuda.empty_cache()
    
    predictions = np.array(predictions)
    difficulties = np.array(difficulties)
    
    print(f"\nAnalyzed {len(predictions)} PSyllabus samples")
    
    # Step 4: Compute correlations between predictions and difficulty
    from scipy.stats import spearmanr, pearsonr
    
    correlations = {}
    print(f"\n{'='*50}")
    print("CORRELATION WITH DIFFICULTY")
    print(f"{'='*50}")
    print(f"{'Dimension':<25} {'Spearman':>10} {'p-value':>10}")
    print("-"*50)
    
    for i, dim in enumerate(PERCEPIANO_DIMENSIONS):
        rho, p = spearmanr(predictions[:, i], difficulties)
        correlations[dim] = {'spearman_rho': float(rho), 'p_value': float(p)}
        sig = '*' if p < 0.05 else ''
        print(f"{dim:<25} {rho:>10.3f} {p:>10.4f} {sig}")
    
    # Overall quality correlation
    overall_pred = predictions.mean(axis=1)
    overall_rho, overall_p = spearmanr(overall_pred, difficulties)
    print("-"*50)
    print(f"{'Overall (mean)':<25} {overall_rho:>10.3f} {overall_p:>10.4f}")
    
    # Save results
    final_results = {
        'experiment_id': exp_id,
        'n_samples': len(predictions),
        'difficulty_range': [int(difficulties.min()), int(difficulties.max())],
        'correlations': correlations,
        'overall_correlation': {'spearman_rho': float(overall_rho), 'p_value': float(overall_p)},
        'by_difficulty': {
            int(d): int((difficulties == d).sum())
            for d in sorted(set(difficulties))
        },
    }
    
    with open(RESULTS_DIR / f'{exp_id}.json', 'w') as f:
        json.dump(final_results, f, indent=2)
    
    sync_experiment_to_gdrive(exp_id, final_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS)
    ALL_RESULTS[exp_id] = final_results
else:
    print(f"\nSKIP {exp_id}: already complete")
    existing = RESULTS_DIR / f'{exp_id}.json'
    if existing.exists():
        with open(existing) as f:
            ALL_RESULTS[exp_id] = json.load(f)


In [ ]:
# C2: ASAP Multi-Performer Analysis
# Analyze prediction variance for same pieces played by different performers
# Uses MAESTRO audio via ASAP mappings
# Requires: MAESTRO_MULTI_PERFORMER from setup cell

exp_id = 'C2_asap_multiperformer'

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: {exp_id}")
    print(f"Description: Multi-performer variance analysis on ASAP/MAESTRO")
    print(f"{'='*70}")
    
    # Check if MAESTRO data is available
    if 'MAESTRO_MULTI_PERFORMER' not in dir() or len(MAESTRO_MULTI_PERFORMER) == 0:
        raise RuntimeError("Run MAESTRO+ASAP setup cell first!")
    
    print(f"Multi-performer pieces available: {len(MAESTRO_MULTI_PERFORMER)}")
    
    # Setup MuQ cache for MAESTRO
    MAESTRO_MUQ_DIR = MAESTRO_ROOT / 'muq_cache'
    MAESTRO_MUQ_DIR.mkdir(parents=True, exist_ok=True)
    
    # Get list of audio files to process
    maestro_audio_files = []
    for title, entries in MAESTRO_MULTI_PERFORMER.items():
        for entry in entries:
            audio_path = MAESTRO_EXTRACTED / entry['audio_filename']
            if audio_path.exists():
                maestro_audio_files.append({
                    'path': audio_path,
                    'key': entry['audio_filename'].replace('.wav', '').replace('/', '_'),
                    'title': title,
                    'split': entry.get('split', 'unknown'),
                })
    
    print(f"Audio files for multi-performer pieces: {len(maestro_audio_files)}")
    
    # Extract MuQ embeddings
    cached_keys = {p.stem for p in MAESTRO_MUQ_DIR.glob('*.pt')}
    missing_keys = [f for f in maestro_audio_files if f['key'] not in cached_keys]
    
    if missing_keys:
        print(f"Extracting {len(missing_keys)} MuQ embeddings...")
        # Create temp symlinks for extraction
        temp_audio_dir = MAESTRO_ROOT / 'temp_audio'
        temp_audio_dir.mkdir(exist_ok=True)
        for f in missing_keys:
            link_path = temp_audio_dir / f"{f['key']}.wav"
            if not link_path.exists():
                link_path.symlink_to(f['path'])
        
        extract_muq_embeddings(
            temp_audio_dir, MAESTRO_MUQ_DIR, [f['key'] for f in missing_keys],
            layer_start=9, layer_end=13
        )
    
    # Load best model
    best_ckpt = None
    ckpt_candidates = [
        CHECKPOINT_ROOT / 'A1c_stratified_fold' / 'fold0_best.ckpt',
        CHECKPOINT_ROOT / 'A1a_piece_fold' / 'fold0_best.ckpt',
    ]
    for ckpt in ckpt_candidates:
        if ckpt.exists():
            best_ckpt = ckpt
            break
    
    if best_ckpt is None:
        raise RuntimeError("No trained model found!")
    
    print(f"Loading model from {best_ckpt.name}...")
    model = MuQStatsModel.load_from_checkpoint(best_ckpt)
    model = model.to('cuda').eval()
    
    # Run inference and group by piece
    piece_predictions = {}
    for f in tqdm(maestro_audio_files, desc="MAESTRO inference"):
        emb_path = MAESTRO_MUQ_DIR / f"{f['key']}.pt"
        if not emb_path.exists():
            continue
        
        with torch.no_grad():
            emb = torch.load(emb_path, weights_only=True).unsqueeze(0).cuda()
            if emb.shape[1] > 300:
                emb = emb[:, :300, :]
            mask = torch.ones(1, emb.shape[1], dtype=torch.bool).cuda()
            pred = model(emb, mask).cpu().numpy()[0]
            
            title = f['title']
            if title not in piece_predictions:
                piece_predictions[title] = []
            piece_predictions[title].append(pred)
    
    del model
    torch.cuda.empty_cache()
    
    # Analyze variance across performers for each piece
    print(f"\n{'='*50}")
    print("MULTI-PERFORMER VARIANCE ANALYSIS")
    print(f"{'='*50}")
    
    intra_piece_stds = []
    piece_stats = {}
    
    for title, preds in piece_predictions.items():
        if len(preds) >= 2:
            preds_array = np.array(preds)
            mean_pred = preds_array.mean(axis=0)
            std_pred = preds_array.std(axis=0)
            
            piece_stats[title] = {
                'n_performances': len(preds),
                'mean_std': float(std_pred.mean()),
                'max_std': float(std_pred.max()),
            }
            intra_piece_stds.append(std_pred.mean())
    
    # Summary statistics
    mean_intra_std = np.mean(intra_piece_stds) if intra_piece_stds else 0
    
    print(f"Pieces with 2+ performances: {len(piece_stats)}")
    print(f"Mean intra-piece std: {mean_intra_std:.4f}")
    print(f"\nInterpretation:")
    if mean_intra_std > 0.1:
        print("  High variance -> Model captures performer-specific interpretation")
    else:
        print("  Low variance -> Model captures piece-invariant features")
    
    # Save results
    final_results = {
        'experiment_id': exp_id,
        'n_pieces': len(piece_stats),
        'n_performances_total': sum(p['n_performances'] for p in piece_stats.values()),
        'mean_intra_piece_std': float(mean_intra_std),
        'piece_stats': piece_stats,
    }
    
    with open(RESULTS_DIR / f'{exp_id}.json', 'w') as f:
        json.dump(final_results, f, indent=2)
    
    sync_experiment_to_gdrive(exp_id, final_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS)
    ALL_RESULTS[exp_id] = final_results
else:
    print(f"\nSKIP {exp_id}: already complete")
    existing = RESULTS_DIR / f'{exp_id}.json'
    if existing.exists():
        with open(existing) as f:
            ALL_RESULTS[exp_id] = json.load(f)


In [ ]:
# C3: MAESTRO Zero-Shot Transfer
# Evaluate model on MAESTRO dataset (no ground truth, analyze prediction distribution)
# Requires: MAESTRO_METADATA from setup cell

exp_id = 'C3_maestro_transfer'

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: {exp_id}")
    print(f"Description: Zero-shot evaluation on MAESTRO dataset")
    print(f"{'='*70}")
    
    # Check if MAESTRO data is available
    if 'MAESTRO_METADATA' not in dir() or len(MAESTRO_METADATA) == 0:
        raise RuntimeError("Run MAESTRO+ASAP setup cell first!")
    
    print(f"MAESTRO entries available: {len(MAESTRO_METADATA)}")
    
    # Setup MuQ cache
    MAESTRO_MUQ_DIR = MAESTRO_ROOT / 'muq_cache'
    MAESTRO_MUQ_DIR.mkdir(parents=True, exist_ok=True)
    
    # Sample subset for analysis (full dataset is large)
    MAX_SAMPLES = 500
    random.seed(42)
    maestro_sample = random.sample(MAESTRO_METADATA, min(MAX_SAMPLES, len(MAESTRO_METADATA)))
    
    # Get audio files
    maestro_files = []
    for entry in maestro_sample:
        audio_path = MAESTRO_EXTRACTED / entry['audio_filename']
        if audio_path.exists():
            maestro_files.append({
                'path': audio_path,
                'key': entry['audio_filename'].replace('.wav', '').replace('/', '_'),
                'canonical_title': entry['canonical_title'],
                'year': entry.get('year', 'unknown'),
                'split': entry.get('split', 'unknown'),
            })
    
    print(f"Sampled {len(maestro_files)} MAESTRO files for analysis")
    
    # Extract MuQ embeddings
    cached_keys = {p.stem for p in MAESTRO_MUQ_DIR.glob('*.pt')}
    missing_keys = [f for f in maestro_files if f['key'] not in cached_keys]
    
    if missing_keys:
        print(f"Extracting {len(missing_keys)} MuQ embeddings...")
        temp_audio_dir = MAESTRO_ROOT / 'temp_audio'
        temp_audio_dir.mkdir(exist_ok=True)
        for f in missing_keys:
            link_path = temp_audio_dir / f"{f['key']}.wav"
            if not link_path.exists():
                link_path.symlink_to(f['path'])
        
        extract_muq_embeddings(
            temp_audio_dir, MAESTRO_MUQ_DIR, [f['key'] for f in missing_keys],
            layer_start=9, layer_end=13
        )
    
    # Load best model
    best_ckpt = None
    ckpt_candidates = [
        CHECKPOINT_ROOT / 'A1c_stratified_fold' / 'fold0_best.ckpt',
        CHECKPOINT_ROOT / 'A1a_piece_fold' / 'fold0_best.ckpt',
    ]
    for ckpt in ckpt_candidates:
        if ckpt.exists():
            best_ckpt = ckpt
            break
    
    if best_ckpt is None:
        raise RuntimeError("No trained model found!")
    
    print(f"Loading model from {best_ckpt.name}...")
    model = MuQStatsModel.load_from_checkpoint(best_ckpt)
    model = model.to('cuda').eval()
    
    # Run inference
    predictions = []
    for f in tqdm(maestro_files, desc="MAESTRO inference"):
        emb_path = MAESTRO_MUQ_DIR / f"{f['key']}.pt"
        if not emb_path.exists():
            continue
        
        with torch.no_grad():
            emb = torch.load(emb_path, weights_only=True).unsqueeze(0).cuda()
            if emb.shape[1] > 300:
                emb = emb[:, :300, :]
            mask = torch.ones(1, emb.shape[1], dtype=torch.bool).cuda()
            pred = model(emb, mask).cpu().numpy()[0]
            predictions.append(pred)
    
    del model
    torch.cuda.empty_cache()
    
    predictions = np.array(predictions)
    
    # Analyze prediction distributions
    print(f"\n{'='*50}")
    print("MAESTRO PREDICTION DISTRIBUTION")
    print(f"{'='*50}")
    print(f"Samples: {len(predictions)}")
    print(f"\n{'Dimension':<25} {'Mean':>8} {'Std':>8} {'Min':>8} {'Max':>8}")
    print("-"*60)
    
    prediction_stats = {}
    for i, dim in enumerate(PERCEPIANO_DIMENSIONS):
        dim_preds = predictions[:, i]
        stats = {
            'mean': float(dim_preds.mean()),
            'std': float(dim_preds.std()),
            'min': float(dim_preds.min()),
            'max': float(dim_preds.max()),
        }
        prediction_stats[dim] = stats
        print(f"{dim:<25} {stats['mean']:>8.3f} {stats['std']:>8.3f} {stats['min']:>8.3f} {stats['max']:>8.3f}")
    
    # Compare to PercePiano distribution
    print(f"\nNote: MAESTRO is professional competition recordings,")
    print(f"      so we expect higher mean predictions than PercePiano training data.")
    
    # Save results
    final_results = {
        'experiment_id': exp_id,
        'n_samples': len(predictions),
        'prediction_stats': prediction_stats,
        'note': 'Zero-shot transfer to MAESTRO (no ground truth labels)',
    }
    
    with open(RESULTS_DIR / f'{exp_id}.json', 'w') as f:
        json.dump(final_results, f, indent=2)
    
    sync_experiment_to_gdrive(exp_id, final_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS)
    ALL_RESULTS[exp_id] = final_results
else:
    print(f"\nSKIP {exp_id}: already complete")
    existing = RESULTS_DIR / f'{exp_id}.json'
    if existing.exists():
        with open(existing) as f:
            ALL_RESULTS[exp_id] = json.load(f)


---
## PHASE D: Advanced Analysis

Detailed analysis of model behavior and per-dimension performance.

In [ ]:
# D1: Per-Dimension R2 Breakdown
# Analyze which of the 19 PercePiano dimensions are best/worst predicted
exp_id = 'D1_per_dimension'

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: {exp_id}")
    print(f"Description: Per-dimension R2 breakdown for 19 PercePiano dimensions")
    print(f"{'='*70}")
    
    # Use the best model from Phase A
    best_exp_id = f"A1{'a' if BEST_FOLD_METHOD == 'piece_based' else 'b' if BEST_FOLD_METHOD == 'performer_based' else 'c'}_{BEST_FOLD_METHOD.replace('_based', '') if 'based' in BEST_FOLD_METHOD else BEST_FOLD_METHOD}_fold"
    best_checkpoint_dir = CHECKPOINT_ROOT / best_exp_id
    
    if not best_checkpoint_dir.exists():
        print(f"Best model checkpoint not found: {best_checkpoint_dir}")
    else:
        # Collect all predictions and labels across folds
        all_preds, all_labels = [], []
        
        for fold in range(4):
            ckpt_path = best_checkpoint_dir / f"fold{fold}_best.ckpt"
            if not ckpt_path.exists():
                continue
            
            model = MuQStatsModel.load_from_checkpoint(ckpt_path)
            model.eval().to("cuda")
            
            val_ds = MERTDataset(MUQ_L9_12_DIR, LABELS, BEST_FOLD_ASSIGNMENTS, fold, "val", M1C_CONFIG["max_frames"])
            val_dl = DataLoader(val_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
                                collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
            
            with torch.no_grad():
                for batch in val_dl:
                    pred = model(batch["embeddings"].cuda(), batch["attention_mask"].cuda(), batch.get("lengths"))
                    all_preds.append(pred.cpu().numpy())
                    all_labels.append(batch["labels"].numpy())
            
            del model
            torch.cuda.empty_cache()
        
        all_preds = np.vstack(all_preds)
        all_labels = np.vstack(all_labels)
        
        # Compute per-dimension R2
        per_dim_r2 = {}
        for i, dim_name in enumerate(PERCEPIANO_DIMENSIONS):
            dim_r2 = r2_score(all_labels[:, i], all_preds[:, i])
            per_dim_r2[dim_name] = float(dim_r2)
        
        # Sort by R2
        sorted_dims = sorted(per_dim_r2.items(), key=lambda x: x[1], reverse=True)
        
        print(f"\n{'Dimension':<30} {'R2':>10} {'Category':<20}")
        print("-"*65)
        for dim_name, r2 in sorted_dims:
            category = DIMENSION_CATEGORIES.get(dim_name, 'Unknown')
            print(f"{dim_name:<30} {r2:>10.4f} {category:<20}")
        
        # Category-level aggregation
        category_r2s = defaultdict(list)
        for dim_name, r2 in per_dim_r2.items():
            cat = DIMENSION_CATEGORIES.get(dim_name, 'Unknown')
            category_r2s[cat].append(r2)
        
        print(f"\n{'Category':<20} {'Mean R2':>10} {'Count':>8}")
        print("-"*40)
        for cat, r2s in sorted(category_r2s.items(), key=lambda x: np.mean(x[1]), reverse=True):
            print(f"{cat:<20} {np.mean(r2s):>10.4f} {len(r2s):>8}")
        
        # Save results
        existing_results_file = RESULTS_DIR / f'{exp_id}.json'
        final_results = {
            'experiment_id': exp_id, 'status': 'complete',
            'per_dimension_r2': per_dim_r2,
            'sorted_dimensions': sorted_dims,
            'category_mean_r2': {cat: float(np.mean(r2s)) for cat, r2s in category_r2s.items()},
            'best_dimensions': sorted_dims[:5],
            'worst_dimensions': sorted_dims[-5:],
        }
        with open(existing_results_file, 'w') as f:
            json.dump(final_results, f, indent=2, default=numpy_serializer)
        
        sync_experiment_to_gdrive(exp_id, final_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS, ALL_RESULTS)
        ALL_RESULTS[exp_id] = final_results
else:
    print(f"\nSKIP {exp_id}: already complete")

In [ ]:
# D2: Error Analysis by Composer, Difficulty, and Segment Position
exp_id = 'D2_error_analysis'

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: {exp_id}")
    print(f"Description: Error analysis by composer, difficulty, segment position")
    print(f"{'='*70}")
    
    def get_composer(key: str) -> str:
        """Extract composer from sample key."""
        parts = key.split('_')
        if len(parts) >= 2:
            return parts[1] if parts[0].startswith('P') else parts[0]
        return 'Unknown'
    
    def get_segment_position(key: str) -> str:
        """Extract segment position from key (early/middle/late)."""
        parts = key.split('_')
        for p in parts:
            if p.startswith('seg'):
                seg_num = int(p.replace('seg', ''))
                if seg_num <= 2:
                    return 'early'
                elif seg_num <= 5:
                    return 'middle'
                else:
                    return 'late'
        return 'unknown'
    
    # Use best model predictions (reuse from D1 if available)
    best_exp_id = f"A1{'a' if BEST_FOLD_METHOD == 'piece_based' else 'b' if BEST_FOLD_METHOD == 'performer_based' else 'c'}_{BEST_FOLD_METHOD.replace('_based', '') if 'based' in BEST_FOLD_METHOD else BEST_FOLD_METHOD}_fold"
    best_checkpoint_dir = CHECKPOINT_ROOT / best_exp_id
    
    if not best_checkpoint_dir.exists():
        print(f"Best model checkpoint not found: {best_checkpoint_dir}")
    else:
        # Collect predictions with keys
        all_preds, all_labels, all_keys = [], [], []
        
        for fold in range(4):
            ckpt_path = best_checkpoint_dir / f"fold{fold}_best.ckpt"
            if not ckpt_path.exists():
                continue
            
            model = MuQStatsModel.load_from_checkpoint(ckpt_path)
            model.eval().to("cuda")
            
            val_ds = MERTDataset(MUQ_L9_12_DIR, LABELS, BEST_FOLD_ASSIGNMENTS, fold, "val", M1C_CONFIG["max_frames"])
            val_dl = DataLoader(val_ds, batch_size=M1C_CONFIG["batch_size"], shuffle=False,
                                collate_fn=mert_collate_fn, num_workers=M1C_CONFIG["num_workers"], pin_memory=True)
            
            with torch.no_grad():
                for batch in val_dl:
                    pred = model(batch["embeddings"].cuda(), batch["attention_mask"].cuda(), batch.get("lengths"))
                    all_preds.append(pred.cpu().numpy())
                    all_labels.append(batch["labels"].numpy())
                    all_keys.extend(batch["keys"])
            
            del model
            torch.cuda.empty_cache()
        
        all_preds = np.vstack(all_preds)
        all_labels = np.vstack(all_labels)
        
        # Compute per-sample errors
        errors = np.abs(all_preds - all_labels).mean(axis=1)  # Mean absolute error per sample
        
        # Analysis by composer
        composer_errors = defaultdict(list)
        for key, err in zip(all_keys, errors):
            composer = get_composer(key)
            composer_errors[composer].append(err)
        
        print("\n--- Error by Composer ---")
        print(f"{'Composer':<20} {'Mean MAE':>10} {'Std':>10} {'N':>8}")
        print("-"*50)
        for composer, errs in sorted(composer_errors.items(), key=lambda x: np.mean(x[1])):
            if len(errs) >= 5:  # Only show composers with enough samples
                print(f"{composer[:20]:<20} {np.mean(errs):>10.4f} {np.std(errs):>10.4f} {len(errs):>8}")
        
        # Analysis by segment position
        position_errors = defaultdict(list)
        for key, err in zip(all_keys, errors):
            pos = get_segment_position(key)
            position_errors[pos].append(err)
        
        print("\n--- Error by Segment Position ---")
        print(f"{'Position':<15} {'Mean MAE':>10} {'Std':>10} {'N':>8}")
        print("-"*45)
        for pos in ['early', 'middle', 'late', 'unknown']:
            if pos in position_errors:
                errs = position_errors[pos]
                print(f"{pos:<15} {np.mean(errs):>10.4f} {np.std(errs):>10.4f} {len(errs):>8}")
        
        # Analysis by label magnitude (difficulty proxy)
        label_means = all_labels.mean(axis=1)
        low_label_mask = label_means < np.percentile(label_means, 33)
        high_label_mask = label_means > np.percentile(label_means, 66)
        
        print("\n--- Error by Label Magnitude ---")
        print(f"{'Label Range':<20} {'Mean MAE':>10} {'N':>8}")
        print("-"*40)
        print(f"{'Low (bottom 33%)':<20} {np.mean(errors[low_label_mask]):>10.4f} {sum(low_label_mask):>8}")
        print(f"{'Medium (middle 33%)':<20} {np.mean(errors[~low_label_mask & ~high_label_mask]):>10.4f} {sum(~low_label_mask & ~high_label_mask):>8}")
        print(f"{'High (top 33%)':<20} {np.mean(errors[high_label_mask]):>10.4f} {sum(high_label_mask):>8}")
        
        # Save results
        existing_results_file = RESULTS_DIR / f'{exp_id}.json'
        final_results = {
            'experiment_id': exp_id, 'status': 'complete',
            'composer_errors': {k: {'mean': float(np.mean(v)), 'std': float(np.std(v)), 'n': len(v)} 
                               for k, v in composer_errors.items() if len(v) >= 5},
            'position_errors': {k: {'mean': float(np.mean(v)), 'std': float(np.std(v)), 'n': len(v)}
                               for k, v in position_errors.items()},
            'label_magnitude_errors': {
                'low': float(np.mean(errors[low_label_mask])),
                'medium': float(np.mean(errors[~low_label_mask & ~high_label_mask])),
                'high': float(np.mean(errors[high_label_mask])),
            },
        }
        with open(existing_results_file, 'w') as f:
            json.dump(final_results, f, indent=2, default=numpy_serializer)
        
        sync_experiment_to_gdrive(exp_id, final_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS, ALL_RESULTS)
        ALL_RESULTS[exp_id] = final_results
else:
    print(f"\nSKIP {exp_id}: already complete")

---
## Final Summary

Generate paper-ready summary of all experiments and upload to GDrive.

This section compiles all Phase A, B, and C results into tables suitable for the paper.

In [ ]:
# Cell 60: Generate paper-ready summary table
print("="*70)
print("STRONGEST PAPER EXPERIMENTS: FINAL SUMMARY")
print("="*70)

print("\n" + "="*70)
print("TIER 1 RESULTS (Critical for acceptance)")
print("="*70)

# Multi-seed stability
if 'multi_seed_stability' in ALL_RESULTS:
    ms = ALL_RESULTS['multi_seed_stability']
    print(f"\n1. Multi-Seed Stability:")
    print(f"   Mean R2: {ms['summary']['mean_r2']:.4f}")
    print(f"   Std R2:  {ms['summary']['std_r2']:.4f}")
    print(f"   Status:  {ms['summary']['stability_status']}")

# Stratified folds
if 'stratified_folds' in ALL_RESULTS:
    sf = ALL_RESULTS['stratified_folds']
    print(f"\n2. Stratified Fold Redistribution:")
    print(f"   Avg R2: {sf['summary']['avg_r2']:.4f}")
    print(f"   Std R2: {sf['summary']['std_r2']:.4f}")
    if 'comparison' in sf:
        print(f"   Variance reduction: {sf['comparison']['variance_reduction_pct']:.1f}%")

# D7 completion
if 'D7_muq_baseline' in ALL_RESULTS:
    d7 = ALL_RESULTS['D7_muq_baseline']
    print(f"\n3. D7_muq_baseline (Complete):")
    print(f"   Avg R2: {d7['summary']['avg_r2']:.4f}")
    print(f"   CI: [{d7['summary']['r2_ci_95'][0]:.4f}, {d7['summary']['r2_ci_95'][1]:.4f}]")

print("\n" + "="*70)
print("TIER 2 RESULTS (Strengthen significantly)")
print("="*70)

# Pianoteq augmentation
if 'pianoteq_augmentation' in ALL_RESULTS:
    pa = ALL_RESULTS['pianoteq_augmentation']
    print(f"\n4. Pianoteq Soundfont Augmentation:")
    print(f"   Status: {pa['status']}")
    if 'soundfonts' in pa:
        for sf, data in pa['soundfonts'].items():
            print(f"     {sf}: {data['count']} files")

# MAESTRO analysis
if 'maestro_analysis' in ALL_RESULTS:
    ma = ALL_RESULTS['maestro_analysis']
    print(f"\n5. MAESTRO Cross-Dataset Analysis:")
    print(f"   Samples: {ma['n_samples']}")

print("\n" + "="*70)
print("TIER 3 RESULTS (Polish and depth)")
print("="*70)

# Error analysis
if 'error_analysis' in ALL_RESULTS:
    ea = ALL_RESULTS['error_analysis']
    print(f"\n6. Error Analysis: Complete")

# Ablations
if 'ablation_studies' in ALL_RESULTS:
    ab = ALL_RESULTS['ablation_studies']
    print(f"\n7. Ablation Studies: {ab['status']}")

In [ ]:
# Cell 61: Save aggregate results
aggregate_results = {
    'notebook': 'train_strongest_paper.ipynb',
    'description': 'Strongest paper experiments for ISMIR 2026',
    'experiments': ALL_RESULTS,
    'verification_checklist': {
        'multi_seed_stability': 'multi_seed_stability' in ALL_RESULTS,
        'stratified_folds': 'stratified_folds' in ALL_RESULTS,
        'd7_complete': 'D7_muq_baseline' in ALL_RESULTS,
        'pianoteq_augmentation': 'pianoteq_augmentation' in ALL_RESULTS,
        'maestro_analysis': 'maestro_analysis' in ALL_RESULTS,
        'error_analysis': 'error_analysis' in ALL_RESULTS,
        'ablation_studies': 'ablation_studies' in ALL_RESULTS,
    },
}

# Save locally
with open(RESULTS_DIR / 'strongest_paper_all_results.json', 'w') as f:
    json.dump(aggregate_results, f, indent=2, default=numpy_serializer)

print(f"Saved aggregate results to {RESULTS_DIR / 'strongest_paper_all_results.json'}")

In [ ]:
# Cell 62: Upload all results to GDrive
print("\nUploading all results to GDrive...")

# Upload aggregate results
run_rclone(['rclone', 'copyto',
            str(RESULTS_DIR / 'strongest_paper_all_results.json'),
            f'{GDRIVE_RESULTS}/strongest_paper_all_results.json'],
           "Uploading aggregate results")

# Upload all individual result files
for json_file in RESULTS_DIR.glob('*.json'):
    run_rclone(['rclone', 'copyto',
                str(json_file),
                f'{GDRIVE_RESULTS}/{json_file.name}'],
               f"Uploading {json_file.name}")

# Upload all checkpoints
if CHECKPOINT_ROOT.exists():
    run_rclone(['rclone', 'copy',
                str(CHECKPOINT_ROOT),
                f'{GDRIVE_RESULTS}/checkpoints',
                '--progress'],
               "Uploading all checkpoints")

print("\n" + "="*70)
print("ALL EXPERIMENTS COMPLETE!")
print("="*70)
print(f"\nResults uploaded to: {GDRIVE_RESULTS}")
print("\nVerification commands:")
print(f"  rclone cat {GDRIVE_RESULTS}/multi_seed_stability.json | python3 -c \"import json,sys; d=json.load(sys.stdin); print(f'Mean R2: {{d[\\\"summary\\\"][\\\"mean_r2\\\"]:.4f}}')\"")
print(f"  rclone cat {GDRIVE_RESULTS}/stratified_folds.json | python3 -c \"import json,sys; d=json.load(sys.stdin); print(f'Avg R2: {{d[\\\"summary\\\"][\\\"avg_r2\\\"]:.4f}}')\"")
print("\n" + "="*70)